v52  
- **ERROR** This model is has a minor error. It requires benchmark ticker (i.e. SPY) to be in the cascade df_ohlcv. As a consequence, the 2nd filter can select SPY, without SPY being passed by the first filter. Bot_v55 fixed this in walk_forward_analyzer.
- **Cascase Filter results `AGREED` with bot_v54i.ipynb**
- **Cascade Filter works with df_ohlcv_subset**
- **verify_engine_results_short_form**
- **verify_engine_results_long_form**
-  **The Temporal Alignment Fix:** We synchronized the "Reward" (Returns) and "Risk" (Volatility) by implementing the $N-1$ denominator logic. This ensures that Day 1's volatility no longer dilutes your Sharpe scores.
-  **The Event-Driven Re-normalization:** We verified that the Engine correctly resets capital and weights at the start of the Holding period, giving you an accurate "Fresh Start" performance metric.
-  **The Double-Blind Verification:** We proved that the Engine's True Range (TRP) math is flawless by recreating it from raw High/Low/Close data and achieving an 8-decimal match.
-  **Mathematical Fortification:** We centralized all logic into a polymorphic `QuantUtils` kernel that handles both single-portfolio reports and whole-universe rankings with built-in numerical safety.
-  **Volatility Evolution:** We successfully added `TRP` (True Range Percent) and the `Sharpe (TRP)` metric, giving you a raw, high-frequency alternative to the smoothed ATR.
-  **Data Integrity:** We implemented the "Momentum Collapse" tripwire (`verify_ranking_integrity`) to ensure that your risk-adjusted rankings never accidentally devolve into simple price momentum.
-  **The "Audit Pack" Architecture:** We collapsed fragmented results into a single, atomic container, ensuring that your inputs, results, and debug data are always perfectly synchronized.
-  **Total Transparency:** We replaced scattered CSV files with a unified **Excel Audit Report**, allowing for 1-to-1 manual verification of every calculation in the system.



v51

UNDO v50, Calculate Sharpe(ATR) using mean over lookback period.  

Comment out ``# --- PINPOINT START: ATRP SWITCH ---`` in function ``_select_tickers`` can switch between ``Averaged ATRP over lookback period`` and ``Current ATRP``  
    # --- PINPOINT START: ATRP SWITCH ---  
    # To switch between Old (Averaged ATRP) and New (Current ATRP):  
    # 1. Comment out the logic you DON'T want.  
    # 2. Uncomment the logic you DO want.  


v50

Ticker selection based on atrp_value_for_obs based on decision day, was based on average over lookback period. 

v48  
### Summary of what you just accomplished:
1.  **Strict Math:** `QuantUtils` now contains an `assert` that prevents any dev (or AI) from filling the first day with 0.0.
2.  **Semantic Protection:** Variables are now named `returns_WITH_BOUNDARY_NAN`, signaling to the AI that the Null value is part of its identity.
3.  **Complete SOLID Separation:** The Engine CONDUCTS the simulation, while `QuantUtils` CALCULATES the results. They no longer share logic.

**1. Data Flow of `plot_walk_forward_analyzer`**
The function acts as a **UI wrapper** around the `AlphaEngine` class. The flow is:
1.  **Input:** User selects parameters (Dates, Lookback, Strategy).
2.  **State Construction:** `AlphaEngine` slices the historical data (`df_ohlcv`, `df_atrp`) up to the `decision_date`.
3.  **Policy Execution (Hardcoded):** The engine applies the logic (e.g., `METRIC_REGISTRY['Sharpe']`) to rank stocks based *only* on the Lookback window.
4.  **Environment Step:** It simulates a "Buy" at `decision_date + 1` and calculates the returns over the `holding_period`.
5.  **Reward Generation:** It outputs performance metrics (`holding_p_gain`, `holding_p_sharpe`).

In [1]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets

from dataclasses import dataclass, field, asdict, is_dataclass
from typing import List, Dict, Optional, Any, Union, TypedDict, Tuple
from collections import Counter
from datetime import datetime, date
from pandas.testing import assert_series_equal


# pd.set_option('display.max_rows', None)  display all rows
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)
pd.set_option("display.max_colwidth", 50)
pd.set_option("display.precision", 4)


# ==============================================================================
# GLOBAL SETTINGS: The "Control Panel" for the Strategy
# ==============================================================================

GLOBAL_SETTINGS = {
    # ENVIRONMENT (The "Where")
    "benchmark_ticker": "SPY",
    "calendar_ticker": "SPY",  # Used as the "Master Clock" for trading days
    # DATA SANITIZER (The "Glitches & Gaps" Protector)
    "handle_zeros_as_nan": True,  # Convert 0.0 prices to NaN to prevent math errors
    "max_data_gap_ffill": 1,  # Max consecutive days to "Forward Fill" missing data
    # IMPLICATION OF nan_price_replacement:
    # - This defines what happens if the "Forward Fill" limit is exceeded.
    # - If set to 0.0: A permanent data gap will look like a "total loss" (-100%).
    #   The equity curve will plummet. Good for "disaster detection."
    #   Sharpe and Sharpe(ATR) drop because: return (gets smaller) / std (gets larger)
    # - If set to np.nan: A permanent gap will cause portfolio calculations to return NaN.
    #   The chart may break or show gaps. Good for "math integrity."
    "nan_price_replacement": 0.0,
    # STRATEGY & MATH
    "annual_period": 252,  # Replaces hardcoded 252 in Sharpe calculations
    "atr_period": 14,  # Used for volatility normalization
    "rsi_period": 14,  # <--- NEW: Control for RSI logic
    # FEATURE ENGINE WINDOWS
    "features_base_window": 21,  # Replaces hardcoded 21 (The "Monthly" anchor)
    "features_fast_window": 5,  # Replaces hardcoded 5 (The "Weekly" anchor)
    # FEATURE GUARDRAILS (CLIPS)
    "feature_zscore_clip": 5.0,  # Replaces hardcoded 5.0 in OBV Z-Scores
    "feature_ratio_clip": 10.0,  # Replaces hardcoded 10.0 in RVol ratios
    # QUALITY/LIQUIDITY
    "quality_window": 252,  # 1 year lookback for liquidity/quality stats
    "quality_min_periods": 126,  # min period that ticker has to meet quality thresholds
    # QUALITY THRESHOLDS (The "Rules")
    "thresholds": {
        # HARD LIQUIDITY FLOOR
        # Logic: Calculates (Adj Close * Volume) daily, then takes the ROLLING MEDIAN
        # over the quality_window (252 days). Filters out stocks where the
        # typical daily dollar turnover is below this absolute value.
        "min_median_dollar_volume": 1_000_000,
        # DYNAMIC LIQUIDITY CUTOFF (Relative to Universe)
        # Logic: On the decision date, the engine calculates the X-quantile
        # of 'RollMedDollarVol' across ALL available stocks.
        # Setting this to 0.40 calculates the 60th percentile and requires
        # stocks to be above it—effectively keeping only the TOP 60% of the market.
        "min_liquidity_percentile": 0.40,
        # PRICE/VOLUME STALENESS
        # Logic: Creates a binary flag (1 if Volume is 0 OR High equals Low).
        # It then calculates the ROLLING MEAN of this flag.
        # A value of 0.05 means the stock is rejected if it was "stale"
        # for more than 5% of the trading days in the rolling window.
        "max_stale_pct": 0.05,
        # DATA INTEGRITY (FROZEN VOLUME)
        # Logic: Checks if Volume is identical to the previous day (Volume.diff() == 0).
        # It calculates the ROLLING SUM of these occurrences over the window.
        # If the exact same volume is reported more than 10 times, the stock
        # is rejected as having "frozen" or low-quality data.
        "max_same_vol_count": 10,
    },
}


# ==============================================================================
# SECTION A: CORE KERNELS & QUANT UTILITIES (THE SAFE ROOM)
# ==============================================================================


class QuantUtils:
    """
    MATHEMATICAL KERNEL REGISTRY: THE SINGLE SOURCE OF TRUTH.
    Handles both pd.Series (Report) and pd.DataFrame (Ranking) robustly.
    """

    @staticmethod
    def compute_returns(
        data: Union[pd.Series, pd.DataFrame],
    ) -> Union[pd.Series, pd.DataFrame]:
        return data.pct_change().replace([np.inf, -np.inf], np.nan)

    @staticmethod
    def calculate_gain(data: Union[pd.Series, pd.DataFrame]) -> Union[float, pd.Series]:
        if data.empty:
            return 0.0
        res = (data.ffill().iloc[-1] / data.bfill().iloc[0]) - 1

        if isinstance(res, (pd.Series, pd.DataFrame)):
            return res.replace([np.inf, -np.inf], np.nan).fillna(0.0)
        return float(res) if np.isfinite(res) else 0.0

    @staticmethod
    def calculate_sharpe(
        data: Union[pd.Series, pd.DataFrame],
        periods: int = None,  # Default to None to trigger global lookup
    ) -> Union[float, pd.Series]:
        if periods is None:
            periods = GLOBAL_SETTINGS["annual_period"]
        mu, std = data.mean(), data.std()
        # Use np.maximum for universal floor (works on scalars and Series)
        res = (mu / np.maximum(std, 1e-8)) * np.sqrt(periods)

        if isinstance(res, (pd.Series, pd.DataFrame)):
            return res.replace([np.inf, -np.inf], np.nan).fillna(0.0)
        return float(res) if np.isfinite(res) else 0.0

    @staticmethod
    def calculate_sharpe_vol(
        returns: Union[pd.Series, pd.DataFrame],
        vol_data: Union[pd.Series, pd.DataFrame],
    ) -> Union[float, pd.Series]:
        """
        Aligned Reward / Risk.
        Filters out volatility observations where no return exists (e.g. Day 1 NaN).
        """
        # 1. Identify valid timestamps (Pandas .mean() skips NaNs in returns)
        # but we must manually force the volatility denominator to skip those same rows.
        mask = returns.notna()
        avg_ret = returns.mean()

        # 2. Handle Logic Branches
        if isinstance(returns, pd.DataFrame) and isinstance(vol_data, pd.Series):
            # RANKING MODE: vol_data is usually a pre-calculated snapshot Series
            avg_vol = vol_data
        else:
            # REPORT MODE (Series) or Cross-Sectional DataFrame
            # Filter vol_data to only include rows where returns exist
            avg_vol = vol_data.where(mask).mean()

        # 3. Final Division
        res = avg_ret / np.maximum(avg_vol, 1e-8)

        if isinstance(res, (pd.Series, pd.DataFrame)):
            return res.replace([np.inf, -np.inf], np.nan).fillna(0.0)
        return float(res) if np.isfinite(res) else 0.0

    @staticmethod
    def compute_portfolio_stats(
        prices: pd.DataFrame,
        atrp_matrix: pd.DataFrame,
        trp_matrix: pd.DataFrame,
        weights: pd.Series,
    ) -> Tuple[pd.Series, pd.Series, pd.Series, pd.Series]:
        """
        MATRIX KERNEL: Calculates equity curve and weighted volatility.
        """
        # 1. Equity Curve Logic (Price-Weighted Drift)
        norm_prices = prices.div(prices.bfill().iloc[0])
        weighted_components = norm_prices.mul(weights, axis=1)
        equity_curve = weighted_components.sum(axis=1)

        # MANDATORY: Use internal compute_returns to preserve boundary NaN
        returns_WITH_BOUNDARY_NAN = QuantUtils.compute_returns(equity_curve)

        # 2. Portfolio Volatility Logic (Weighted Average)
        # We calculate current_weights (rebalanced daily by price drift)
        current_weights = weighted_components.div(equity_curve, axis=0)

        # Weighted average of ATRP and TRP
        portfolio_atrp = (current_weights * atrp_matrix).sum(axis=1, min_count=1)
        portfolio_trp = (current_weights * trp_matrix).sum(axis=1, min_count=1)

        return equity_curve, returns_WITH_BOUNDARY_NAN, portfolio_atrp, portfolio_trp


# ==============================================================================
# SECTION B: STRATEGY HELPERS & FEATURES
# ==============================================================================


def generate_features(
    df_ohlcv: pd.DataFrame,
    df_indices: pd.DataFrame = None,
    benchmark_ticker: str = None,
    atr_period: int = None,
    rsi_period: int = None,
    features_base_window: int = None,
    features_fast_window: int = None,
    feature_zscore_clip: float = None,
    feature_ratio_clip: float = None,
    quality_window: int = None,
    quality_min_periods: int = None,
) -> pd.DataFrame:
    # --- 1. RESOLVE GLOBALS ---
    benchmark_ticker = benchmark_ticker or GLOBAL_SETTINGS["benchmark_ticker"]
    atr_period = atr_period or GLOBAL_SETTINGS["atr_period"]
    rsi_period = rsi_period or GLOBAL_SETTINGS["rsi_period"]
    features_base_window = (
        features_base_window or GLOBAL_SETTINGS["features_base_window"]
    )
    features_fast_window = (
        features_fast_window or GLOBAL_SETTINGS["features_fast_window"]
    )
    feature_zscore_clip = feature_zscore_clip or GLOBAL_SETTINGS["feature_zscore_clip"]
    feature_ratio_clip = feature_ratio_clip or GLOBAL_SETTINGS["feature_ratio_clip"]
    quality_window = quality_window or GLOBAL_SETTINGS["quality_window"]
    quality_min_periods = quality_min_periods or GLOBAL_SETTINGS["quality_min_periods"]

    print(
        f"⚡ Generating Features (Base: {features_base_window}d, Ratio Clip: {feature_ratio_clip})..."
    )

    if not df_ohlcv.index.is_monotonic_increasing:
        df_ohlcv = df_ohlcv.sort_index()
    grouped = df_ohlcv.groupby(level="Ticker")

    # 2. VECTORIZED ATR
    prev_close = grouped["Adj Close"].shift(1)
    tr = pd.concat(
        [
            df_ohlcv["Adj High"] - df_ohlcv["Adj Low"],
            abs(df_ohlcv["Adj High"] - prev_close),
            abs(df_ohlcv["Adj Low"] - prev_close),
        ],
        axis=1,
    ).max(axis=1, skipna=False)

    atr = (
        tr.groupby(level="Ticker")
        .ewm(alpha=1 / atr_period, adjust=False)
        .mean()
        .reset_index(level=0, drop=True)
    )
    atrp = (atr / df_ohlcv["Adj Close"]).replace([np.inf, -np.inf], np.nan)
    trp = (tr / df_ohlcv["Adj Close"]).replace([np.inf, -np.inf], np.nan)

    # 3. VECTORIZED RSI
    delta = grouped["Adj Close"].diff()
    up, down = delta.clip(lower=0), -1 * delta.clip(upper=0)
    ma_up = (
        up.groupby(level="Ticker")
        .ewm(alpha=1 / rsi_period, adjust=False)
        .mean()
        .reset_index(level=0, drop=True)
    )
    ma_down = (
        down.groupby(level="Ticker")
        .ewm(alpha=1 / rsi_period, adjust=False)
        .mean()
        .reset_index(level=0, drop=True)
    )
    rsi = 100 - (100 / (1 + (ma_up / ma_down))).replace([np.inf, -np.inf], 50).fillna(
        50
    )

    # 4. OBV Score (Ticker Specific)
    direction = np.sign(delta).fillna(0)
    obv_raw = (direction * df_ohlcv["Volume"]).groupby(level="Ticker").cumsum()
    obv_roll_mean = (
        obv_raw.groupby(level="Ticker")
        .rolling(features_base_window)
        .mean()
        .reset_index(level=0, drop=True)
    )
    obv_roll_std = (
        obv_raw.groupby(level="Ticker")
        .rolling(features_base_window)
        .std()
        .reset_index(level=0, drop=True)
    )
    obv_score = (
        ((obv_raw - obv_roll_mean) / obv_roll_std)
        .fillna(0.0)
        .clip(lower=-feature_zscore_clip, upper=feature_zscore_clip)
    )

    # 5. BENCHMARK SENSORS
    dollar_vol_series = df_ohlcv["Adj Close"] * df_ohlcv["Volume"]
    rel_strength_base = pd.Series(0.0, index=df_ohlcv.index)
    spy_rvol = pd.Series(1.0, index=df_ohlcv.index)
    spy_obv_score = pd.Series(0.0, index=df_ohlcv.index)

    found_bench = False
    if (
        df_indices is not None
        and benchmark_ticker in df_indices.index.get_level_values(0)
    ):
        bench_close = df_indices.xs(benchmark_ticker, level=0)["Adj Close"]
        bench_vol = df_indices.xs(benchmark_ticker, level=0)["Volume"]
        found_bench = True
    elif benchmark_ticker in df_ohlcv.index.get_level_values(0):
        bench_close = df_ohlcv.xs(benchmark_ticker, level=0)["Adj Close"]
        bench_vol = df_ohlcv.xs(benchmark_ticker, level=0)["Volume"]
        found_bench = True

    if found_bench:
        try:
            # Relative Strength (Using Base Window)
            bench_close_aligned = bench_close.reindex(
                df_ohlcv.index.get_level_values("Date")
            ).values
            rel_strength_base = (
                (df_ohlcv["Adj Close"] / bench_close_aligned)
                .groupby(level="Ticker")
                .pct_change(features_base_window, fill_method=None)
                .fillna(0.0)
            )

            # SPY RVol
            bench_dvol = bench_close * bench_vol
            bench_dvol_avg = bench_dvol.rolling(features_base_window).mean()
            spy_rvol_vals = (
                (bench_dvol / bench_dvol_avg)
                .reindex(df_ohlcv.index.get_level_values("Date"))
                .fillna(1.0)
                .values
            )
            spy_rvol = pd.Series(spy_rvol_vals, index=df_ohlcv.index).clip(
                upper=feature_ratio_clip
            )

            # SPY OBV
            spy_dir = np.sign(bench_close.diff()).fillna(0)
            spy_obv = (spy_dir * bench_vol).cumsum()
            spy_obv_z = (
                (
                    (spy_obv - spy_obv.rolling(features_base_window).mean())
                    / spy_obv.rolling(features_base_window).std()
                )
                .fillna(0.0)
                .clip(lower=-feature_zscore_clip, upper=feature_zscore_clip)
            )
            spy_obv_score = pd.Series(
                spy_obv_z.reindex(df_ohlcv.index.get_level_values("Date")).values,
                index=df_ohlcv.index,
            )
        except Exception as e:
            print(f"⚠️ Benchmark Math Error: {e}")

    # 6. TICKER RVol
    dvol_avg = (
        dollar_vol_series.groupby(level="Ticker")
        .rolling(features_base_window)
        .mean()
        .reset_index(level=0, drop=True)
    )
    ticker_rvol = (
        (dollar_vol_series / dvol_avg)
        .replace([np.inf, -np.inf], 1.0)
        .fillna(1.0)
        .clip(upper=feature_ratio_clip)
    )

    # 7. MOMENTUM VECTORS (Static as requested)
    roc_1 = grouped["Adj Close"].pct_change(1, fill_method=None)
    roc_3 = grouped["Adj Close"].pct_change(3, fill_method=None)
    roc_5 = grouped["Adj Close"].pct_change(5, fill_method=None)
    roc_10 = grouped["Adj Close"].pct_change(10, fill_method=None)
    roc_21 = grouped["Adj Close"].pct_change(21, fill_method=None)

    # 8. VOLATILITY REGIME (Fast / Base)
    rets = grouped["Adj Close"].pct_change(1, fill_method=None)
    std_fast = (
        rets.groupby(level="Ticker")
        .rolling(features_fast_window)
        .std()
        .reset_index(level=0, drop=True)
    )
    std_base = (
        rets.groupby(level="Ticker")
        .rolling(features_base_window)
        .std()
        .reset_index(level=0, drop=True)
    )
    vol_regime = (std_fast / std_base).replace([np.inf, -np.inf], 1.0)

    # 9. OUTPUT
    indicator_df = pd.DataFrame(
        {
            "ATR": atr,
            "ATRP": atrp,
            "TRP": trp,
            "RSI": rsi,
            "RelStrength": rel_strength_base,
            "VolRegime": vol_regime,
            "RVol": ticker_rvol,
            "Spy_RVol": spy_rvol,
            "OBV_Score": obv_score,
            "Spy_OBV_Score": spy_obv_score,
            "ROC_1": roc_1,
            "ROC_3": roc_3,
            "ROC_5": roc_5,
            "ROC_10": roc_10,
            "ROC_21": roc_21,
        }
    )

    # 10. QUALITY/LIQUIDITY
    quality_temp = pd.DataFrame(
        {
            "IsStale": np.where(
                (df_ohlcv["Volume"] == 0)
                | (df_ohlcv["Adj High"] == df_ohlcv["Adj Low"]),
                1,
                0,
            ),
            "DollarVolume": dollar_vol_series,
            "HasSameVolume": (grouped["Volume"].diff() == 0).astype(int),
        },
        index=df_ohlcv.index,
    )

    rolling_quality = (
        quality_temp.groupby(level="Ticker")
        .rolling(window=quality_window, min_periods=quality_min_periods)
        .agg({"IsStale": "mean", "DollarVolume": "median", "HasSameVolume": "sum"})
        .rename(
            columns={
                "IsStale": "RollingStalePct",
                "DollarVolume": "RollMedDollarVol",
                "HasSameVolume": "RollingSameVolCount",
            }
        )
        .reset_index(level=0, drop=True)
    )

    return pd.concat([indicator_df, rolling_quality], axis=1)


def _prepare_initial_weights(tickers: List[str]) -> pd.Series:
    """
    METADATA: Converts a list of tickers into a weight map.
    Example: ['AAPL', 'AAPL', 'TSLA'] -> {'AAPL': 0.66, 'TSLA': 0.33}
    """
    ticker_counts = Counter(tickers)
    total = len(tickers)
    return pd.Series({t: c / total for t, c in ticker_counts.items()})


def calculate_buy_and_hold_performance(
    df_close_wide: pd.DataFrame,  # Use the WIDE version
    df_atrp_wide: pd.DataFrame,  # Use the WIDE version
    df_trp_wide: pd.DataFrame,  # <--- Added
    tickers: List[str],
    start_date: pd.Timestamp,
    end_date: pd.Timestamp,
):
    if not tickers:
        return pd.Series(), pd.Series(), pd.Series()

    initial_weights = _prepare_initial_weights(tickers)

    # SLICE (Fix Part B)
    ticker_list = initial_weights.index.tolist()
    p_slice = df_close_wide.reindex(columns=ticker_list).loc[start_date:end_date]
    a_slice = df_atrp_wide.reindex(columns=ticker_list).loc[start_date:end_date]
    t_slice = df_trp_wide.reindex(columns=ticker_list).loc[start_date:end_date]
    # KERNEL - Pure Math
    return QuantUtils.compute_portfolio_stats(
        p_slice, a_slice, t_slice, initial_weights
    )


# ==============================================================================
# SECTION C: METRIC REGISTRY
# ==============================================================================


class MarketObservation(TypedDict):
    """
    The 'STATE' (Observation) in Reinforcement Learning.
    This defines the context given to the agent to make a decision.
    """

    # --- The Movie (Time Series) ---
    lookback_returns: pd.DataFrame  # (Time x Tickers)
    lookback_close: pd.DataFrame  # (Time x Tickers)

    # --- The Snapshot (Scalar values at Decision Time) ---
    atrp: pd.Series  # Volatility (Mean over lookback)
    trp: pd.Series  # Volatility (Snapshot)

    # NEW SENSORS
    rsi: pd.Series  # Internal Momentum (0-100)
    rel_strength: pd.Series  # Performance vs SPY
    vol_regime: pd.Series  # Volatility Expansion/Compression
    rvol: pd.Series  # Ticker Conviction
    spy_rvol: pd.Series  # Market Participation
    obv_score: pd.Series  # Ticker Accumulation/Distribution
    spy_obv_score: pd.Series  # Market Tide

    # MOMENTUM VECTORS
    roc_1: pd.Series
    roc_3: pd.Series
    roc_5: pd.Series
    roc_10: pd.Series
    roc_21: pd.Series


METRIC_REGISTRY = {
    # --- CLASSIC METRICS ---
    "Price": lambda obs: QuantUtils.calculate_gain(obs["lookback_close"]),
    "Sharpe": lambda obs: QuantUtils.calculate_sharpe(obs["lookback_returns"]),
    "Sharpe (ATRP)": lambda obs: QuantUtils.calculate_sharpe_vol(
        obs["lookback_returns"], obs["atrp"]
    ),
    "Sharpe (TRP)": lambda obs: QuantUtils.calculate_sharpe_vol(
        obs["lookback_returns"], obs["trp"]
    ),  # <--- New Strategy
    # --- MOMENTUM VECTORS ---
    "Momentum 1D": lambda obs: obs["roc_1"],
    "Momentum 3D": lambda obs: obs["roc_3"],
    "Momentum 5D": lambda obs: obs["roc_5"],
    "Momentum 10D": lambda obs: obs["roc_10"],
    "Momentum 1M": lambda obs: obs["roc_21"],
    # --- PULLBACK VECTORS ---
    "Pullback 1D": lambda obs: -obs["roc_1"],
    "Pullback 3D": lambda obs: -obs["roc_3"],
    "Pullback 5D": lambda obs: -obs["roc_5"],
    "Pullback 10D": lambda obs: -obs["roc_10"],
    "Pullback 1M": lambda obs: -obs["roc_21"],
    # --- NEW SOTA SENSORS ---
    "RSI (Reversal)": lambda obs: -obs["rsi"],  # Rank Low RSI (Oversold) higher
    "RSI (Trend)": lambda obs: obs["rsi"],  # Rank High RSI (Strong Trend) higher
    "Alpha (RelStrength)": lambda obs: obs["rel_strength"],  # Rank stocks beating SPY
    "OBV_Score (Accumulation)": lambda obs: obs["obv_score"],  # Rank High OBV Score
    "$V Inc (Cur$V / Avg$V21)": lambda obs: obs["rvol"],  # Rank High Relative Volume
    "Vol Inc (5dStDev / 21dStDev)": lambda obs: obs[
        "vol_regime"
    ],  # Rank High Volatility Expansion
}


# ==============================================================================
# SECTION D: DATA CONTRACTS
# ==============================================================================


@dataclass
class EngineInput:
    mode: str
    start_date: pd.Timestamp
    lookback_period: int
    holding_period: int
    metric: str
    benchmark_ticker: str
    rank_start: int = 1
    rank_end: int = 10
    # Default factory pulls from Global thresholds
    quality_thresholds: Dict[str, float] = field(
        default_factory=lambda: GLOBAL_SETTINGS["thresholds"].copy()
    )
    manual_tickers: List[str] = field(default_factory=list)
    debug: bool = False
    universe_subset: Optional[List[str]] = None


@dataclass
class EngineOutput:
    # 1. CORE DATA (Required - No Defaults)
    portfolio_series: pd.Series
    benchmark_series: pd.Series
    normalized_plot_data: pd.DataFrame
    tickers: List[str]
    initial_weights: pd.Series
    perf_metrics: Dict[str, float]
    results_df: pd.DataFrame

    # 2. TIMELINE (Required - No Defaults)
    start_date: pd.Timestamp
    decision_date: pd.Timestamp
    buy_date: pd.Timestamp
    holding_end_date: pd.Timestamp

    # 3. OPTIONAL / AUDIT DATA (Must be at the bottom because they have defaults)
    portfolio_atrp_series: Optional[pd.Series] = None
    benchmark_atrp_series: Optional[pd.Series] = None
    portfolio_trp_series: Optional[pd.Series] = None
    benchmark_trp_series: Optional[pd.Series] = None
    error_msg: Optional[str] = None
    debug_data: Optional[Dict[str, Any]] = None


class AlphaEngine:
    def __init__(
        self,
        df_ohlcv: pd.DataFrame,
        features_df: pd.DataFrame = None,
        df_close_wide: pd.DataFrame = None,
        df_atrp_wide: pd.DataFrame = None,
        df_trp_wide: pd.DataFrame = None,
        master_ticker: str = GLOBAL_SETTINGS["calendar_ticker"],
    ):
        print("--- ⚙️ Initializing AlphaEngine v2.2 (Sanitized) ---")

        # 1. SETUP PRICES (CLEAN-AT-ENTRY)
        if df_close_wide is not None:
            self.df_close = df_close_wide
        else:
            print("🐢 Pivoting and Sanitizing Price Data...")
            self.df_close = df_ohlcv["Adj Close"].unstack(level=0)

        # STORE RAW SOURCE (The "Transparency" Line)
        self.df_ohlcv_raw = df_ohlcv

        # 3. DATA SANITIZER STEP 1: Handle Zeros
        if GLOBAL_SETTINGS["handle_zeros_as_nan"]:
            # Replace 0.0 with NaN so math functions (mean/std) ignore them
            self.df_close = self.df_close.replace(0, np.nan)

        # Smooth over 1-2 day glitches (The "FNV" Fix)
        self.df_close = self.df_close.ffill(limit=GLOBAL_SETTINGS["max_data_gap_ffill"])

        # Handle the remaining "unfillable" gaps
        self.df_close = self.df_close.fillna(GLOBAL_SETTINGS["nan_price_replacement"])

        # 2. SETUP FEATURES
        if features_df is not None:
            self.features_df = features_df
        else:
            # We pass the cleaned price data if needed, or calculate from raw
            self.features_df = generate_features(
                df_ohlcv,
                atr_period=GLOBAL_SETTINGS["atr_period"],
                quality_window=GLOBAL_SETTINGS["quality_window"],
                quality_min_periods=GLOBAL_SETTINGS["quality_min_periods"],
            )

        # 1. SETUP ATRP
        if df_atrp_wide is not None:
            # INSTANT: Use the matrix precomputed outside the UI
            self.df_atrp = df_atrp_wide
        else:
            # SLOW FALLBACK: Only runs if you forget to precompute
            print("🚀 Pre-aligning Volatility (ATRP) Matrix (Slow Fallback)...")
            self.df_atrp = self.features_df["ATRP"].unstack(level=0)

        # 2. SETUP TRP
        if df_trp_wide is not None:
            self.df_trp = df_trp_wide
        else:
            print("🚀 Pre-aligning Volatility (TRP) Matrix (Slow Fallback)...")
            self.df_trp = self.features_df["TRP"].unstack(level=0)

        # 3. FINAL ALIGNMENT (The "Safety Seal")
        # Ensures all matrices have the exact same Dimensions, Tickers, and Dates
        common_idx = self.df_close.index
        common_cols = self.df_close.columns

        self.df_atrp = self.df_atrp.reindex(index=common_idx, columns=common_cols)
        self.df_trp = self.df_trp.reindex(index=common_idx, columns=common_cols)

        # 3. Setup Calendar
        if master_ticker not in self.df_close.columns:
            master_ticker = self.df_close.columns[0]
        self.trading_calendar = (
            self.df_close[master_ticker].dropna().index.unique().sort_values()
        )

    def run(self, inputs: EngineInput) -> EngineOutput:
        dates, error = self._validate_timeline(inputs)
        if error:
            return self._error_result(error)
        (safe_start, safe_decision, safe_buy, safe_end) = dates

        # 1. Pass the debug flag into the selection logic
        tickers_to_trade, results_table, debug_dict, error = self._select_tickers(
            inputs, safe_start, safe_decision
        )
        if error:
            return self._error_result(error)

        # 2. CORE PERFORMANCE CALCULATION (Required for all modes)
        p_f_val, p_f_ret, p_f_atrp, p_f_trp = calculate_buy_and_hold_performance(
            self.df_close,
            self.df_atrp,
            self.df_trp,
            tickers_to_trade,
            safe_start,
            safe_end,
        )
        b_f_val, b_f_ret, b_f_atrp, b_f_trp = calculate_buy_and_hold_performance(
            self.df_close,
            self.df_atrp,
            self.df_trp,
            [inputs.benchmark_ticker],
            safe_start,
            safe_end,
        )

        p_h_val, p_h_ret, p_h_atrp, p_h_trp = calculate_buy_and_hold_performance(
            self.df_close,
            self.df_atrp,
            self.df_trp,
            tickers_to_trade,
            safe_buy,
            safe_end,
        )
        b_h_val, b_h_ret, b_h_atrp, b_h_trp = calculate_buy_and_hold_performance(
            self.df_close,
            self.df_atrp,
            self.df_trp,
            [inputs.benchmark_ticker],
            safe_buy,
            safe_end,
        )

        p_metrics, p_slices = self._calculate_period_metrics(
            p_f_val,
            p_f_ret,
            p_f_atrp,
            p_f_trp,
            safe_decision,
            p_h_val,
            p_h_ret,
            p_h_atrp,
            p_h_trp,
            prefix="p",
        )
        b_metrics, b_slices = self._calculate_period_metrics(
            b_f_val,
            b_f_ret,
            b_f_atrp,
            b_f_trp,
            safe_decision,
            b_h_val,
            b_h_ret,
            b_h_atrp,
            b_h_trp,
            prefix="b",
        )

        # 3. CONDITIONAL DEBUG GATING (The RL "Fast Path")
        # If debug is False, we skip the most memory-intensive tasks
        final_debug_data = None
        if inputs.debug:
            idx_slice = pd.IndexSlice
            debug_dict["inputs_snapshot"] = inputs
            debug_dict["verification"] = {"portfolio": p_slices, "benchmark": b_slices}

            debug_dict["portfolio_raw_components"] = {
                "prices": self.df_close[tickers_to_trade].loc[safe_start:safe_end],
                "atrp": self.df_atrp[tickers_to_trade].loc[safe_start:safe_end],
                "trp": self.df_trp[tickers_to_trade].loc[safe_start:safe_end],
                "ohlcv_raw": self.df_ohlcv_raw.loc[
                    idx_slice[tickers_to_trade, safe_start:safe_end], :
                ],
            }

            debug_dict["benchmark_raw_components"] = {
                "prices": self.df_close[[inputs.benchmark_ticker]].loc[
                    safe_start:safe_end
                ],
                "atrp": self.df_atrp[[inputs.benchmark_ticker]].loc[
                    safe_start:safe_end
                ],
                "trp": self.df_trp[[inputs.benchmark_ticker]].loc[safe_start:safe_end],
                "ohlcv_raw": self.df_ohlcv_raw.loc[
                    idx_slice[[inputs.benchmark_ticker], safe_start:safe_end], :
                ],
            }
            debug_dict["selection_audit"] = debug_dict.get("full_universe_ranking")
            final_debug_data = debug_dict

        # 4. FINAL OUTPUT SEAL
        return EngineOutput(
            portfolio_series=p_f_val,
            benchmark_series=b_f_val,
            portfolio_atrp_series=p_f_atrp if inputs.debug else None,
            benchmark_atrp_series=b_f_atrp if inputs.debug else None,
            portfolio_trp_series=p_f_trp if inputs.debug else None,
            benchmark_trp_series=b_f_trp if inputs.debug else None,
            normalized_plot_data=(
                self._get_normalized_plot_data(tickers_to_trade, safe_start, safe_end)
                if inputs.debug
                else pd.DataFrame()
            ),
            tickers=tickers_to_trade,
            initial_weights=_prepare_initial_weights(tickers_to_trade),
            perf_metrics={**p_metrics, **b_metrics},
            results_df=results_table,
            start_date=safe_start,
            decision_date=safe_decision,
            buy_date=safe_buy,
            holding_end_date=safe_end,
            debug_data=final_debug_data,  # None if RL search mode
        )

    # ==============================================================================
    # INTERNAL LOGIC MODULES
    # ==============================================================================

    def _validate_timeline(self, inputs: EngineInput):
        cal = self.trading_calendar
        last_idx = len(cal) - 1

        if len(cal) <= inputs.lookback_period:
            return (
                None,
                f"❌ Dataset too small.\nNeed > {inputs.lookback_period} days of history.",
            )

        # 2. Check "Past" Constraints (Lookback)
        min_decision_date = cal[inputs.lookback_period]
        if inputs.start_date < min_decision_date:
            # Added \n here
            return None, (
                f"❌ Not enough history for a {inputs.lookback_period}-day lookback.\n"
                f"Earliest valid Decision Date: {min_decision_date.date()}"
            )

        # 3. Check "Future" Constraints (Entry T+1 and Holding Period)
        required_future_days = 1 + inputs.holding_period
        latest_valid_idx = last_idx - required_future_days

        if latest_valid_idx < 0:
            return (
                None,
                f"❌ Holding period too long.\n{inputs.holding_period} days exceeds available data.",
            )

        # If user picked a date beyond the available "future" runway
        if inputs.start_date > cal[latest_valid_idx]:
            latest_date = cal[latest_valid_idx].date()
            # Added \n here and shortened the text slightly to fit better
            return None, (
                f"❌ Decision Date too late for a {inputs.holding_period}-day hold.\n"
                f"Latest valid date: {latest_date}. Please move picker back."
            )

        # 4. Map the safe indices
        decision_idx = cal.searchsorted(inputs.start_date)
        if decision_idx > latest_valid_idx:
            decision_idx = latest_valid_idx

        start_idx = decision_idx - inputs.lookback_period
        entry_idx = decision_idx + 1
        end_idx = entry_idx + inputs.holding_period

        return (cal[start_idx], cal[decision_idx], cal[entry_idx], cal[end_idx]), None

    def _select_tickers(self, inputs: EngineInput, start_date, decision_date):
        debug_dict = {}

        # --- PATH A: MANUAL LIST ---
        if inputs.mode == "Manual List":
            validation_errors = []
            valid_tickers = []
            for t in inputs.manual_tickers:
                if t not in self.df_close.columns:
                    validation_errors.append(f"❌ {t}: Not found.")
                    continue
                if pd.isna(self.df_close.at[start_date, t]):
                    validation_errors.append(f"⚠️ {t}: No data on start date.")
                    continue
                valid_tickers.append(t)

            if validation_errors:
                return [], pd.DataFrame(), {}, "\n".join(validation_errors)
            if not valid_tickers:
                return [], pd.DataFrame(), {}, "No valid tickers found."
            return valid_tickers, pd.DataFrame(index=valid_tickers), {}, None

        # --- PATH B: RANKING ---
        else:
            audit_info = {}
            eligible_tickers = self._filter_universe(
                decision_date, inputs.quality_thresholds, audit_info
            )
            debug_dict["audit_liquidity"] = audit_info

            if not eligible_tickers:
                return (
                    [],
                    pd.DataFrame(),
                    debug_dict,
                    "No tickers passed quality filters.",
                )

            lookback_close = self.df_close.loc[
                start_date:decision_date, eligible_tickers
            ]

            # 1. Get the Snapshot of Features for the Decision Date
            feat_slice_current = self.features_df.xs(
                decision_date, level="Date"
            ).reindex(eligible_tickers)

            # Calculate mean ATRP over the lookback period
            idx_product = pd.MultiIndex.from_product(
                [eligible_tickers, lookback_close.index], names=["Ticker", "Date"]
            )
            feat_slice_period = self.features_df.reindex(idx_product)
            atrp_value_for_obs = (
                feat_slice_period["ATRP"].groupby(level="Ticker").mean()
            )

            # --- PINPOINT CHANGE: Calculate mean TRP over lookback ---
            trp_value_for_obs = feat_slice_period["TRP"].groupby(level="Ticker").mean()

            # Update the observation dictionary
            observation: MarketObservation = {
                # ...
                "atrp": atrp_value_for_obs,
                "trp": trp_value_for_obs,  # <--- PINPOINT CHANGE: Pass the lookback mean
                # ...
            }

            # 2. Package the Observation (The 'State')
            observation: MarketObservation = {
                # Time Series Data
                "lookback_close": lookback_close,
                "lookback_returns": lookback_close.ffill().pct_change(),
                # Snapshot Data (Scalar values for today)
                "atrp": atrp_value_for_obs,  # <--- USES THE TOGGLED VALUE HERE
                "trp": trp_value_for_obs,  # <--- PINPOINT CHANGE: Pass the lookback mean
                "rsi": feat_slice_current["RSI"],
                "rel_strength": feat_slice_current["RelStrength"],
                "vol_regime": feat_slice_current["VolRegime"],
                "rvol": feat_slice_current["RVol"],
                "spy_rvol": feat_slice_current["Spy_RVol"],
                "obv_score": feat_slice_current["OBV_Score"],
                "spy_obv_score": feat_slice_current["Spy_OBV_Score"],
                # Momentum Vectors
                "roc_1": feat_slice_current["ROC_1"],
                "roc_3": feat_slice_current["ROC_3"],
                "roc_5": feat_slice_current["ROC_5"],
                "roc_10": feat_slice_current["ROC_10"],
                "roc_21": feat_slice_current["ROC_21"],
            }

            # 3. Run the Strategy (The 'Agent')
            if inputs.metric not in METRIC_REGISTRY:
                return [], pd.DataFrame(), {}, f"Strategy '{inputs.metric}' not found."

            metric_vals = METRIC_REGISTRY[inputs.metric](observation)
            sorted_tickers = metric_vals.sort_values(ascending=False)
            start_r = max(0, inputs.rank_start - 1)
            end_r = inputs.rank_end
            selected_tickers = sorted_tickers.iloc[start_r:end_r].index.tolist()

            # Audit
            debug_dict["full_universe_ranking"] = pd.DataFrame(
                {
                    "Strategy_Score": metric_vals,
                    "Lookback_Return_Ann": observation["lookback_returns"].mean() * 252,
                    "Lookback_ATRP": observation["atrp"],
                }
            )

            if not selected_tickers:
                return (
                    [],
                    pd.DataFrame(),
                    debug_dict,
                    "No tickers generated from ranking.",
                )

            results_table = pd.DataFrame(
                {
                    "Rank": range(
                        inputs.rank_start, inputs.rank_start + len(selected_tickers)
                    ),
                    "Ticker": selected_tickers,
                    "Strategy Value": sorted_tickers.loc[selected_tickers].values,
                }
            ).set_index("Ticker")

            return selected_tickers, results_table, debug_dict, None

    def _filter_universe(self, date_ts, thresholds, audit_container=None):
        avail_dates = (
            self.features_df.index.get_level_values("Date").unique().sort_values()
        )
        valid_dates = avail_dates[avail_dates <= date_ts]
        if valid_dates.empty:
            return []
        target_date = valid_dates[-1]
        day_features = self.features_df.xs(target_date, level="Date")

        vol_cutoff = thresholds.get("min_median_dollar_volume", 0)
        percentile_used = "N/A"
        if "min_liquidity_percentile" in thresholds:
            percentile_used = thresholds["min_liquidity_percentile"]
            dynamic_val = day_features["RollMedDollarVol"].quantile(percentile_used)
            vol_cutoff = max(vol_cutoff, dynamic_val)

        mask = (
            (day_features["RollMedDollarVol"] >= vol_cutoff)
            & (day_features["RollingStalePct"] <= thresholds["max_stale_pct"])
            & (day_features["RollingSameVolCount"] <= thresholds["max_same_vol_count"])
        )

        if audit_container is not None:
            audit_container["date"] = target_date
            audit_container["total_tickers_available"] = len(day_features)
            audit_container["percentile_setting"] = percentile_used
            audit_container["final_cutoff_usd"] = vol_cutoff
            audit_container["tickers_passed"] = mask.sum()
            snapshot = day_features.copy()
            snapshot["Calculated_Cutoff"] = vol_cutoff
            snapshot["Passed_Vol_Check"] = snapshot["RollMedDollarVol"] >= vol_cutoff
            snapshot["Passed_Final"] = mask
            snapshot = snapshot.sort_values("RollMedDollarVol", ascending=False)
            audit_container["universe_snapshot"] = snapshot

        return day_features[mask].index.tolist()

    def _calculate_period_metrics(
        self,
        f_val,
        f_ret,
        f_atrp,
        f_trp,
        decision_date,
        h_val,
        h_ret,
        h_atrp,
        h_trp,
        prefix,  # <--- Added trp args
    ):
        metrics = {}
        slices = {}

        # 1. Temporal Slicing (Routing)
        lb_val, lb_ret, lb_atrp, lb_trp = (
            f_val.loc[:decision_date],
            f_ret.loc[:decision_date],
            f_atrp.loc[:decision_date],
            f_trp.loc[:decision_date],
        )

        # Use the new unified QuantUtils calls
        metrics[f"full_{prefix}_gain"] = QuantUtils.calculate_gain(f_val)
        metrics[f"full_{prefix}_sharpe"] = QuantUtils.calculate_sharpe(f_ret)
        metrics[f"full_{prefix}_sharpe_atrp"] = QuantUtils.calculate_sharpe_vol(
            f_ret, f_atrp
        )
        metrics[f"full_{prefix}_sharpe_trp"] = QuantUtils.calculate_sharpe_vol(
            f_ret, f_trp
        )

        metrics[f"lookback_{prefix}_gain"] = QuantUtils.calculate_gain(lb_val)
        metrics[f"lookback_{prefix}_sharpe"] = QuantUtils.calculate_sharpe(lb_ret)
        metrics[f"lookback_{prefix}_sharpe_atrp"] = QuantUtils.calculate_sharpe_vol(
            lb_ret, lb_atrp
        )
        metrics[f"lookback_{prefix}_sharpe_trp"] = QuantUtils.calculate_sharpe_vol(
            lb_ret, lb_trp
        )

        metrics[f"holding_{prefix}_gain"] = QuantUtils.calculate_gain(h_val)
        metrics[f"holding_{prefix}_sharpe"] = QuantUtils.calculate_sharpe(h_ret)
        metrics[f"holding_{prefix}_sharpe_atrp"] = QuantUtils.calculate_sharpe_vol(
            h_ret, h_atrp
        )
        metrics[f"holding_{prefix}_sharpe_trp"] = QuantUtils.calculate_sharpe_vol(
            h_ret, h_trp
        )

        # 5. Metadata Collection
        (
            slices["full_val"],
            slices["full_ret"],
            slices["full_atrp"],
            slices["full_trp"],
        ) = (
            f_val,
            f_ret,
            f_atrp,
            f_trp,
        )
        (
            slices["lookback_val"],
            slices["lookback_ret"],
            slices["lookback_atrp"],
            slices["lookback_trp"],
        ) = (
            lb_val,
            lb_ret,
            lb_atrp,
            lb_trp,
        )
        (
            slices["holding_val"],
            slices["holding_ret"],
            slices["holding_atrp"],
            slices["holding_trp"],
        ) = (
            h_val,
            h_ret,
            h_atrp,
            h_trp,
        )

        return metrics, slices

    def _get_normalized_plot_data(self, tickers, start_date, end_date):
        if not tickers:
            return pd.DataFrame()
        data = self.df_close[list(set(tickers))].loc[start_date:end_date]
        if data.empty:
            return pd.DataFrame()
        return data / data.bfill().iloc[0]

    def _error_result(self, msg):
        return EngineOutput(
            portfolio_series=pd.Series(dtype=float),
            benchmark_series=pd.Series(dtype=float),
            normalized_plot_data=pd.DataFrame(),
            tickers=[],
            initial_weights=pd.Series(dtype=float),
            perf_metrics={},
            results_df=pd.DataFrame(),
            start_date=pd.Timestamp.min,
            decision_date=pd.Timestamp.min,
            buy_date=pd.Timestamp.min,
            holding_end_date=pd.Timestamp.min,
            error_msg=msg,
        )


# ==============================================================================
# SECTION E: THE UI (Visualization)
# ==============================================================================


def plot_walk_forward_analyzer(
    df_ohlcv=None,
    features_df=None,
    df_close_wide=None,
    df_atrp_wide=None,
    df_trp_wide=None,
    engine=None,
    universe_subset=None,  # Logic integrated
    default_start_date="2025-01-17",
    default_lookback=10,
    default_holding=5,
    default_strategy="Sharpe (ATRP)",
    default_rank_start=1,
    default_rank_end=10,
    default_benchmark_ticker=GLOBAL_SETTINGS["benchmark_ticker"],
    master_calendar_ticker=GLOBAL_SETTINGS["calendar_ticker"],
    quality_thresholds=GLOBAL_SETTINGS["thresholds"],
    debug=True,
):

    # 1. THE PERSISTENT ENGINE GATE
    if engine is None:
        if df_ohlcv is None:
            raise ValueError(
                "❌ No engine provided AND no df_ohlcv provided to build one."
            )
        engine = AlphaEngine(
            df_ohlcv,
            features_df=features_df,
            df_close_wide=df_close_wide,
            df_atrp_wide=df_atrp_wide,
            df_trp_wide=df_trp_wide,
            master_ticker=master_calendar_ticker,
        )

    audit_pack = [None]

    if quality_thresholds is None:
        quality_thresholds = GLOBAL_SETTINGS["thresholds"]

    # --- Widgets ---
    mode_selector = widgets.RadioButtons(
        options=["Ranking", "Manual List"],
        value="Ranking",
        description="Mode:",
        layout={"width": "max-content"},
        style={"description_width": "initial"},
    )
    lookback_input = widgets.IntText(
        value=default_lookback,
        description="Lookback (Days):",
        layout={"width": "200px"},
        style={"description_width": "initial"},
    )
    decision_date_picker = widgets.DatePicker(
        description="Decision Date:",
        value=pd.to_datetime(default_start_date),
        layout={"width": "auto"},
        style={"description_width": "initial"},
    )
    holding_input = widgets.IntText(
        value=default_holding,
        description="Holding (Days):",
        layout={"width": "200px"},
        style={"description_width": "initial"},
    )
    strategy_dropdown = widgets.Dropdown(
        options=list(METRIC_REGISTRY.keys()),
        value=default_strategy,
        description="Strategy:",
        layout={"width": "220px"},
        style={"description_width": "initial"},
    )
    benchmark_input = widgets.Text(
        value=default_benchmark_ticker,
        description="Benchmark:",
        placeholder="Enter Ticker",
        layout={"width": "180px"},
        style={"description_width": "initial"},
    )
    rank_start_input = widgets.IntText(
        value=default_rank_start,
        description="Rank Start:",
        layout={"width": "150px"},
        style={"description_width": "initial"},
    )
    rank_end_input = widgets.IntText(
        value=default_rank_end,
        description="Rank End:",
        layout={"width": "150px"},
        style={"description_width": "initial"},
    )
    manual_tickers_input = widgets.Textarea(
        value="",
        placeholder="Enter tickers...",
        description="Manual Tickers:",
        layout={"width": "400px", "height": "80px"},
        style={"description_width": "initial"},
    )
    update_button = widgets.Button(description="Run Simulation", button_style="primary")
    ticker_list_output = widgets.Output()

    # --- Layouts ---
    timeline_box = widgets.HBox(
        [lookback_input, decision_date_picker, holding_input],
        layout=widgets.Layout(
            justify_content="space-between",
            border="1px solid #ddd",
            padding="10px",
            margin="5px",
        ),
    )
    strategy_box = widgets.HBox([strategy_dropdown, benchmark_input])
    ranking_box = widgets.HBox([rank_start_input, rank_end_input])

    def on_mode_change(c):
        ranking_box.layout.display = "flex" if c["new"] == "Ranking" else "none"
        manual_tickers_input.layout.display = (
            "none" if c["new"] == "Ranking" else "flex"
        )
        strategy_dropdown.disabled = c["new"] == "Manual List"

    mode_selector.observe(on_mode_change, names="value")
    on_mode_change({"new": mode_selector.value})

    ui = widgets.VBox(
        [
            widgets.HTML(
                "<b>1. Timeline Configuration:</b> (Past <--- Decision ---> Future)"
            ),
            timeline_box,
            widgets.HTML("<b>2. Strategy Settings:</b>"),
            widgets.HBox([mode_selector, strategy_box]),
            ranking_box,
            manual_tickers_input,
            widgets.HTML("<hr>"),
            update_button,
            ticker_list_output,
        ]
    )

    fig = go.FigureWidget()
    fig.update_layout(
        title="Event-Driven Walk-Forward Analysis",
        height=600,
        template="plotly_white",
        hovermode="x unified",
    )
    for i in range(50):
        fig.add_trace(go.Scatter(visible=False, line=dict(width=2)))
    fig.add_trace(
        go.Scatter(
            name="Benchmark",
            visible=True,
            line=dict(color="black", width=3, dash="dash"),
        )
    )
    fig.add_trace(
        go.Scatter(
            name="Group Portfolio", visible=True, line=dict(color="green", width=3)
        )
    )

    # --- Update Logic ---
    def update_plot(b):
        ticker_list_output.clear_output()
        manual_list = [
            t.strip().upper()
            for t in manual_tickers_input.value.split(",")
            if t.strip()
        ]
        decision_date_raw = pd.to_datetime(decision_date_picker.value)

        # PINPOINT CHANGE: universe_subset now passed to EngineInput
        inputs = EngineInput(
            mode=mode_selector.value,
            start_date=decision_date_raw,
            lookback_period=lookback_input.value,
            holding_period=holding_input.value,
            metric=strategy_dropdown.value,
            benchmark_ticker=benchmark_input.value.strip().upper(),
            rank_start=rank_start_input.value,
            rank_end=rank_end_input.value,
            quality_thresholds=quality_thresholds,
            manual_tickers=manual_list,
            universe_subset=universe_subset,  # <--- Integrated from V2
            debug=debug,
        )

        with ticker_list_output:
            res = engine.run(inputs)
            audit_pack[0] = res

            if res.error_msg:
                print(f"⚠️ Simulation Stopped: {res.error_msg}")
                return

            if not res.normalized_plot_data.empty:
                with fig.batch_update():
                    cols = res.normalized_plot_data.columns.tolist()
                    for i in range(50):
                        if i < len(cols):
                            fig.data[i].update(
                                x=res.normalized_plot_data.index,
                                y=res.normalized_plot_data[cols[i]],
                                name=cols[i],
                                visible=True,
                            )
                        else:
                            fig.data[i].visible = False

                    fig.data[50].update(
                        x=res.benchmark_series.index,
                        y=res.benchmark_series.values,
                        name=f"Benchmark ({inputs.benchmark_ticker})",
                        visible=not res.benchmark_series.empty,
                    )
                    fig.data[51].update(
                        x=res.portfolio_series.index,
                        y=res.portfolio_series.values,
                        visible=True,
                    )

                    fig.layout.shapes = [
                        dict(
                            type="line",
                            x0=res.decision_date,
                            y0=0,
                            x1=res.decision_date,
                            y1=1,
                            xref="x",
                            yref="paper",
                            line=dict(color="red", width=2, dash="dash"),
                        ),
                        dict(
                            type="line",
                            x0=res.buy_date,
                            y0=0,
                            x1=res.buy_date,
                            y1=1,
                            xref="x",
                            yref="paper",
                            line=dict(color="blue", width=2, dash="dot"),
                        ),
                    ]

                start_date = res.start_date.date()
                act_date = res.decision_date.date()
                entry_date = res.buy_date.date()

            # --- Status & Liquidity Output ---
            subset_status = f" | Subset Mode: {'ACTIVE (' + str(len(universe_subset)) + ' tickers)' if universe_subset else 'OFF (Full Market)'}"
            print(f"✅ Success{subset_status}")

            if (
                inputs.mode == "Ranking"
                and res.debug_data
                and "audit_liquidity" in res.debug_data
            ):
                audit = res.debug_data["audit_liquidity"]
                if audit:
                    print("-" * 60)
                    print(f"🔍 LIQUIDITY CHECK (On Decision Date: {act_date})")
                    print(f"   Tickers Remaining: {audit.get('tickers_passed')}")
                    print("-" * 60)

            print(
                f"Timeline: Start [ {start_date} ] --> Decision [ {act_date} ] --> Cash (1d) --> Entry [ {entry_date} ] --> End [ {res.holding_end_date.date()} ]"
            )

            print(f"Tickers ({len(res.tickers)}):")
            for i in range(0, len(res.tickers), 10):
                print(", ".join(res.tickers[i : i + 10]))

            # --- Metrics Table (Full, Lookback, Holding) ---
            m = res.perf_metrics
            metrics_to_show = [
                ("Gain", "gain"),
                ("Sharpe", "sharpe"),
                ("Sharpe (ATRP)", "sharpe_atrp"),
                ("Sharpe (TRP)", "sharpe_trp"),
            ]

            rows = []
            for label, key in metrics_to_show:
                p_row = {
                    "Metric": f"Group {label}",
                    "Full": m.get(f"full_p_{key}"),
                    "Lookback": m.get(f"lookback_p_{key}"),
                    "Holding": m.get(f"holding_p_{key}"),
                }
                b_row = {
                    "Metric": f"Benchmark {label}",
                    "Full": m.get(f"full_b_{key}"),
                    "Lookback": m.get(f"lookback_b_{key}"),
                    "Holding": m.get(f"holding_b_{key}"),
                }
                d_row = {"Metric": f"== {label} Delta"}
                for col in ["Full", "Lookback", "Holding"]:
                    d_row[col] = (p_row[col] or 0) - (b_row[col] or 0)
                rows.extend([p_row, b_row, d_row])

            df_report = pd.DataFrame(rows).set_index("Metric")

            def apply_sleek_style(styler):
                styler.format("{:+.4f}", na_rep="N/A")

                def row_logic(row):
                    if "Delta" in row.name:
                        return [
                            "background-color: #f9f9f9; font-weight: 600; border-top: 1px solid #ddd"
                        ] * len(row)
                    if "Group" in row.name:
                        return ["color: #2c5e8f; background-color: #fcfdfe"] * len(row)
                    return ["color: #555"] * len(row)

                styler.apply(row_logic, axis=1)
                styler.set_table_styles(
                    [
                        {
                            "selector": "",
                            "props": [
                                ("font-family", "inherit"),
                                ("font-size", "12px"),
                                ("border-collapse", "collapse"),
                                ("width", "auto"),
                            ],
                        },
                        {
                            "selector": "th",
                            "props": [
                                ("background-color", "white"),
                                ("color", "#222"),
                                ("font-weight", "600"),
                                ("padding", "6px 12px"),
                                ("border-bottom", "2px solid #444"),
                                ("text-align", "center"),
                            ],
                        },
                        {
                            "selector": "th.row_heading",
                            "props": [
                                ("text-align", "left"),
                                ("padding-right", "30px"),
                                ("border-bottom", "1px solid #eee"),
                            ],
                        },
                        {
                            "selector": "td",
                            "props": [
                                ("padding", "4px 12px"),
                                ("border-bottom", "1px solid #eee"),
                            ],
                        },
                    ]
                )
                styler.index.name = None
                return styler

            display(apply_sleek_style(df_report.style))

    update_button.on_click(update_plot)
    update_plot(None)
    display(ui, fig)
    return audit_pack, engine


# ==============================================================================
# SECTION F: INSPECTION TOOLS
# ==============================================================================

from dataclasses import is_dataclass, fields
import pandas as pd


# ==============================================================================
# INTEGRITY PROTECTION: THE TRIPWIRE
# ==============================================================================


def verify_math_integrity():
    """
    🛡️ TRIPWIRE: Ensures Sample Boundary Integrity.
    """
    print("\n--- 🛡️ Starting Final Integrity Audit ---")

    try:
        # Test 1: Series Input
        mock_series = pd.Series([100.0, 102.0, 101.0])
        rets_s = QuantUtils.compute_returns(mock_series)
        # Verify first value is actually NaN
        if not pd.isna(rets_s.iloc[0]):
            raise ValueError("Series Leading NaN missing")
        print("✅ Series Boundary: OK")

        # Test 2: DataFrame Input
        mock_df = pd.DataFrame({"A": [100, 101], "B": [200, 202]})
        rets_df = QuantUtils.compute_returns(mock_df)
        if not rets_df.iloc[0].isna().all():
            raise ValueError("DataFrame Leading NaN missing")
        print("✅ DataFrame Boundary: OK")

        print("✅ AUDIT PASSED: Mathematical boundaries are strictly enforced.")
    except Exception as e:
        print(f"🔥 SYSTEM BREACH: {str(e)}")
        raise e


def verify_feature_engineering_integrity():
    """
    🛡️ TRIPWIRE: Validates Feature Engineering Logic.
    Enforces:
    1. Day 1 ATR must be NaN (No PrevClose).
    2. Wilder's Smoothing must use Alpha = 1/Period.
    3. Recursion must match manual calculation.
    """
    print("\n--- 🛡️ Starting Feature Engineering Audit ---")

    # 1. Create Synthetic Data (3 Days)
    # Day 1: High-Low = 10. No PrevClose.
    # Day 2: High-Low = 20. Gap up implies TR might be larger.
    # Day 3: High-Low = 10.
    dates = pd.to_datetime(["2020-01-01", "2020-01-02", "2020-01-03"])
    idx = pd.MultiIndex.from_product([["TEST"], dates], names=["Ticker", "Date"])

    df_mock = pd.DataFrame(
        {
            "Adj Open": [100, 110, 110],
            "Adj High": [110, 130, 120],
            "Adj Low": [100, 110, 110],
            "Adj Close": [105, 120, 115],  # PrevClose: NaN, 105, 120
            "Volume": [1000, 1000, 1000],
        },
        index=idx,
    )

    # 2. Run the Generator
    # We use Period=2 to make manual math easy (Alpha = 1/2 = 0.5)
    feats = generate_features(
        df_mock, atr_period=2, rsi_period=2, quality_min_periods=1
    )
    atr_series = feats["ATR"]

    # 3. MANUAL CALCULATION (The "Truth")
    # Day 1:
    #   TR = Max(H-L, |H-PC|, |L-PC|)
    #   TR = Max(10, NaN, NaN) -> NaN (Because skipna=False)
    #   Expected ATR: NaN

    # Day 2:
    #   PrevClose = 105
    #   H-L=20, |130-105|=25, |110-105|=5
    #   TR = 25
    #   Expected ATR: First valid observation = 25.0

    # Day 3:
    #   PrevClose = 120
    #   H-L=10, |120-120|=0, |110-120|=10
    #   TR = 10
    #   Wilder's Smoothing (Alpha=0.5):
    #   ATR_3 = (TR_3 * alpha) + (ATR_2 * (1-alpha))
    #   ATR_3 = (10 * 0.5) + (25 * 0.5) = 5 + 12.5 = 17.5

    print(f"Audit Values:\n{atr_series.values}")

    # 4. ASSERTIONS
    try:
        # Check Day 1
        if not np.isnan(atr_series.iloc[0]):
            raise AssertionError(
                f"Day 1 Regression: Expected NaN, got {atr_series.iloc[0]}. (Check skipna=False)"
            )

        # Check Day 2 (Initialization)
        if not np.isclose(atr_series.iloc[1], 25.0):
            raise AssertionError(
                f"Initialization Regression: Expected 25.0, got {atr_series.iloc[1]}."
            )

        # Check Day 3 (Recursion)
        if not np.isclose(atr_series.iloc[2], 17.5):
            raise AssertionError(
                f"Wilder's Logic Regression: Expected 17.5, got {atr_series.iloc[2]}. (Check Alpha=1/N)"
            )

        print("✅ FEATURE INTEGRITY PASSED: Wilder's ATR logic is strictly enforced.")

    except AssertionError as e:
        print(f"🔥 LOGIC FAILURE: {str(e)}")
        raise e


def verify_ranking_integrity():
    """
    🛡️ TRIPWIRE: Prevents 'Momentum Collapse' in Volatility-Adjusted Ranking.
    Ensures that Sharpe(Vol) distinguishes between High-Vol and Low-Vol stocks.
    """
    print("--- 🛡️ Starting Ranking Kernel Audit ---")

    # 1. Setup Mock Universe (2 Tickers, 2 Days)
    # Ticker 'VOLATILE': 10% return, but 10% Volatility
    # Ticker 'STABLE': 2% return, but 1% Volatility (The 'Sharpe' Winner)
    data = {"VOLATILE": [1.0, 1.10], "STABLE": [1.0, 1.02]}  # +10%  # +2%
    df_returns = pd.DataFrame(data).pct_change().dropna()

    # Pre-calculated Mean Volatility per ticker (as provided by Engine Observation)
    vol_series = pd.Series({"VOLATILE": 0.10, "STABLE": 0.01})

    # 2. Run Kernel
    results = QuantUtils.calculate_sharpe_vol(df_returns, vol_series)

    # 3. CALCULATE EXPECTED (Pure Math)
    # Volatile Sharpe: 0.10 / 0.10 = 1.0
    # Stable Sharpe:   0.02 / 0.01 = 2.0

    try:
        # Check A: Diversity. If they are the same, normalization didn't happen.
        if np.isclose(results["VOLATILE"], results["STABLE"]):
            raise AssertionError(
                "RANKING COLLAPSE: Both tickers have the same normalized score."
            )

        # Check B: Direction. STABLE must rank higher than VOLATILE.
        if results["STABLE"] < results["VOLATILE"]:
            # This is exactly what happens when the bug turns it into Momentum
            raise AssertionError(
                f"MOMENTUM REGRESSION: 'STABLE' ({results['STABLE']:.2f}) "
                f"ranked below 'VOLATILE' ({results['VOLATILE']:.2f}). "
                "The denominator was likely collapsed to a market average."
            )

        # Check C: Absolute Precision
        if not np.isclose(results["STABLE"], 2.0):
            raise AssertionError(
                f"MATH ERROR: Expected 2.0 for STABLE, got {results['STABLE']}"
            )

        print(
            "✅ RANKING INTEGRITY PASSED: Volatility normalization is strictly enforced."
        )

    except Exception as e:
        print(f"🔥 KERNEL BREACH: {str(e)}")
        raise e


def verify_vol_alignment_integrity():
    """
    🛡️ TRIPWIRE: Verifies Temporal Coupling between Returns and Volatility.
    Ensures that the volatility average is only calculated over days where a return exists.
    """
    print("\n--- 🛡️ Starting Volatility Alignment Audit ---")

    # 1. SETUP SYNTHETIC DATA (2 Days)
    # Day 1: Return = NaN, Vol = 0.90 (Extreme 'Trap' Volatility)
    # Day 2: Return = 0.10, Vol = 0.10 (Target Reward/Risk)
    rets_s = pd.Series([np.nan, 0.10])
    vol_s = pd.Series([0.90, 0.10])

    # 2. RUN KERNEL (Series Mode)
    # Calculation Logic:
    # If aligned: 0.10 / 0.10 = 1.0
    # If misaligned: 0.10 / mean(0.90, 0.10) = 0.10 / 0.50 = 0.2
    res_series = QuantUtils.calculate_sharpe_vol(rets_s, vol_s)

    # 3. RUN KERNEL (DataFrame Mode)
    # Ensures vectorized alignment works across columns
    rets_df = pd.DataFrame({"A": [np.nan, 0.10], "B": [np.nan, 0.20]})
    vol_df = pd.DataFrame({"A": [0.90, 0.10], "B": [0.05, 0.20]})
    res_df = QuantUtils.calculate_sharpe_vol(rets_df, vol_df)

    try:
        # Check Series Alignment
        if not np.isclose(res_series, 1.0):
            raise AssertionError(
                f"DENOMINATOR MISMATCH: Series result {res_series:.2f} != 1.0. "
                "The volatility denominator is likely including the leading NaN day."
            )
        print("✅ Series Temporal Coupling: OK")

        # Check DataFrame Alignment (Ticker A: 0.1/0.1=1.0 | Ticker B: 0.2/0.2=1.0)
        if not (np.isclose(res_df["A"], 1.0) and np.isclose(res_df["B"], 1.0)):
            raise AssertionError(
                f"VECTORIZED MISMATCH: DataFrame results {res_df.values} != [1.0, 1.0]. "
                "The logic is failing to align individual columns."
            )
        print("✅ DataFrame Temporal Coupling: OK")

        print("✅ AUDIT PASSED: Reward and Risk are strictly synchronized.")

    except Exception as e:
        print(f"🔥 ALIGNMENT BREACH: {str(e)}")
        raise e


# Auto-run the checks
verify_math_integrity()

verify_feature_engineering_integrity()

verify_ranking_integrity()

verify_vol_alignment_integrity()

#


--- 🛡️ Starting Final Integrity Audit ---
✅ Series Boundary: OK
✅ DataFrame Boundary: OK
✅ AUDIT PASSED: Mathematical boundaries are strictly enforced.

--- 🛡️ Starting Feature Engineering Audit ---
⚡ Generating Features (Base: 21d, Ratio Clip: 10.0)...
Audit Values:
[ nan 25.  17.5]
✅ FEATURE INTEGRITY PASSED: Wilder's ATR logic is strictly enforced.
--- 🛡️ Starting Ranking Kernel Audit ---
✅ RANKING INTEGRITY PASSED: Volatility normalization is strictly enforced.

--- 🛡️ Starting Volatility Alignment Audit ---
✅ Series Temporal Coupling: OK
✅ DataFrame Temporal Coupling: OK
✅ AUDIT PASSED: Reward and Risk are strictly synchronized.


In [2]:
# ==============================================================================
# SECTION G: AUDIT ENGINE RESULTS
# ==============================================================================


def visualize_audit_structure(obj):
    """
    Generates the Map and returns a Registry of dictionaries:
    [{'name': str, 'path': str, 'obj': object}, ...]
    """
    id_memory = {}
    registry = []
    output = [
        "====================================================================",
        "🔍 HIGH-TRANSPARENCY AUDIT MAP",
        "====================================================================",
    ]

    def get_icon(val):
        if isinstance(val, pd.DataFrame):
            return "🧮"
        if isinstance(val, pd.Series):
            return "📈"
        if isinstance(val, (list, tuple, dict)):
            return "📂"
        if isinstance(val, pd.Timestamp):
            return "📅"
        if is_dataclass(val):
            return "📦"
        return "🔢" if isinstance(val, (int, float)) else "📄"

    def process(item, name, level=0, path=""):
        indent = "  " * level
        item_id = id(item)

        # Build the breadcrumb path
        current_path = f"{path} -> {name}" if path else name

        is_primitive = isinstance(item, (int, float, str, bool, type(None)))
        if not is_primitive and item_id in id_memory:
            output.append(
                f"{indent}          ╰── {name} --> [See ID {id_memory[item_id]}]"
            )
            return

        # 1. Store Index, Object, Name, and Path in Registry
        curr_idx = len(registry)
        registry.append({"name": name, "path": current_path, "obj": item})

        if not is_primitive:
            id_memory[item_id] = curr_idx

        # 2. Metadata for display
        meta = f"{type(item).__name__}"
        if hasattr(item, "shape"):
            meta = f"shape={item.shape}"
        elif isinstance(item, (list, dict)):
            meta = f"len={len(item)}"

        output.append(f"[{curr_idx:>3}] {indent}{get_icon(item)} {name} ({meta})")

        # 3. Recurse
        if isinstance(item, dict):
            for k, v in item.items():
                process(v, k, level + 1, current_path)
        elif isinstance(item, (list, tuple)):
            for i, v in enumerate(item):
                process(v, f"index_{i}", level + 1, current_path)
        elif is_dataclass(item):
            for f in fields(item):
                process(getattr(item, f.name), f.name, level + 1, current_path)

    process(obj, "audit_pack")
    print("\n".join(output))

    return registry


def peek(idx, reg):
    """
    Displays metadata and RETURNS the object for further use.
    """
    if idx < 0 or idx >= len(reg):
        print(f"❌ Index {idx} out of range.")
        return None

    entry = reg[idx]

    # 1. Print the Header (for humans)
    print(f" {'='*60}")
    print(f" 📍 INDEX: [{idx}]")
    print(f" 🏷️  NAME:  {entry['name']}")
    print(f" 📂 PATH:  {entry['path']}")
    print(f" {'='*60}\n")

    # 2. Display the data (for the UI)
    from IPython.display import display

    display(entry["obj"])

    # 3. RETURN the data (for other functions)
    return entry["obj"]


def verify_engine_results_short_form(audit_pack, engine):  # <--- Added engine here
    """
    MASTER INTEGRITY REPORT: Performs a 3-layer reconciliation
    between Engine Output and Raw Data.
    """
    # --------------------------------------------------------------------------
    # 0. INITIALIZATION & CONTEXT
    # --------------------------------------------------------------------------
    res = audit_pack[0]
    if not res or res.debug_data is None:
        print("❌ AUDIT ABORTED: No debug data found. Run UI with debug=True.")
        return

    debug = res.debug_data
    inputs = debug["inputs_snapshot"]
    thresholds = inputs.quality_thresholds

    # --- TRANSPARENCY BLOCK ---
    print(
        f"🕵️  STARTING MULTI-LAYER AUDIT: {inputs.metric} @ {res.decision_date.date()}"
    )
    print("\n" + "=" * 85)
    print("*" * 85)
    print(
        f"⚠️  ASSUMPTION: Verification logic is independent, but trusts Engine source DataFrames"
    )
    print(
        f"   (engine.features_df, engine.df_close, and debug['portfolio_raw_components'])"
    )
    print("*" * 85 + "\n" + "=" * 85 + "\n" * 2)

    # --------------------------------------------------------------------------
    # LAYER 1: SURVIVAL AUDIT
    # --------------------------------------------------------------------------
    l_audit = debug["audit_liquidity"]
    snapshot = l_audit["universe_snapshot"]

    m_quantile = snapshot["RollMedDollarVol"].quantile(
        thresholds["min_liquidity_percentile"]
    )
    m_cutoff = max(m_quantile, thresholds["min_median_dollar_volume"])

    m_survivors = snapshot[
        (snapshot["RollMedDollarVol"] >= m_cutoff)
        & (snapshot["RollingStalePct"] <= thresholds["max_stale_pct"])
        & (snapshot["RollingSameVolCount"] <= thresholds["max_same_vol_count"])
    ]

    s_status = "✅ PASS" if len(m_survivors) == l_audit["tickers_passed"] else "❌ FAIL"
    print(
        f"LAYER 1: SURVIVAL  | Universe: {len(snapshot)} -> Survivors: {len(m_survivors)} | {s_status}"
    )

    # --------------------------------------------------------------------------
    # LAYER 2: SELECTION AUDIT (Ranking & Scoring)
    # --------------------------------------------------------------------------
    survivor_list = m_survivors.index.tolist()
    idx = pd.IndexSlice

    # Independent Data Reconstruction using the passed 'engine'
    feat_period = engine.features_df.loc[
        idx[survivor_list, res.start_date : res.decision_date], :
    ]
    atrp_mean = feat_period["ATRP"].groupby(level="Ticker").mean()
    trp_mean = feat_period["TRP"].groupby(level="Ticker").mean()

    feat_now = engine.features_df.xs(res.decision_date, level="Date").reindex(
        survivor_list
    )

    # Check if df_close or df_close_wide is the correct attribute on your engine
    # Based on your Error it was looking for engine.df_close
    lookback_prices = engine.df_close.loc[
        res.start_date : res.decision_date, survivor_list
    ]

    # BUILD COMPLETE OBSERVATION
    audit_obs = {
        "lookback_close": lookback_prices,
        "lookback_returns": lookback_prices.ffill().pct_change(),
        "atrp": atrp_mean,
        "trp": trp_mean,
        "rsi": feat_now["RSI"],
        "rel_strength": feat_now["RelStrength"],
        "vol_regime": feat_now["VolRegime"],
        "rvol": feat_now["RVol"],
        "obv_score": feat_now["OBV_Score"],
        "roc_1": feat_now["ROC_1"],
        "roc_3": feat_now["ROC_3"],
        "roc_5": feat_now["ROC_5"],
        "roc_10": feat_now["ROC_10"],
        "roc_21": feat_now["ROC_21"],
    }

    # Re-run Strategy Formula
    manual_scores = METRIC_REGISTRY[inputs.metric](audit_obs)

    # Comparison logic
    audit_results = []
    for rank, ticker in enumerate(res.tickers, start=inputs.rank_start):
        eng_score = res.results_df.loc[ticker, "Strategy Value"]
        audit_score = manual_scores.loc[ticker]
        audit_results.append(
            {
                "Rank": rank,
                "Ticker": ticker,
                "Engine": eng_score,
                "Manual": audit_score,
                "Delta": eng_score - audit_score,
                "Match": np.isclose(eng_score, audit_score),
            }
        )

    sel_df = pd.DataFrame(audit_results)
    sel_status = "✅ PASS" if sel_df["Match"].all() else "❌ FAIL"
    print(
        f"LAYER 2: SELECTION | Strategy: {inputs.metric} | Selection Match: {sel_status}"
    )

    # --------------------------------------------------------------------------
    # LAYER 3: PERFORMANCE AUDIT (Returns & Risk)
    # --------------------------------------------------------------------------
    def run_kernel(df_p, df_atrp, df_trp, w):
        norm = df_p.div(df_p.bfill().iloc[0])
        eq = (norm * w).sum(axis=1)
        drift_w = (norm * w).div(eq, axis=0)
        p_atrp = (drift_w * df_atrp).sum(axis=1)
        rets = eq.pct_change()
        v_mask = rets.notna()
        g = eq.iloc[-1] / eq.iloc[0] - 1
        s = (rets.mean() / rets.std() * np.sqrt(252)) if rets.std() > 0 else 0
        sa = rets.mean() / p_atrp.where(v_mask).mean()
        return g, s, sa

    p_comp = debug["portfolio_raw_components"]

    m_gain, m_sharpe, m_s_atrp = run_kernel(
        p_comp["prices"].loc[res.buy_date : res.holding_end_date],
        p_comp["atrp"].loc[res.buy_date : res.holding_end_date],
        p_comp["trp"].loc[res.buy_date : res.holding_end_date],
        res.initial_weights,
    )

    e_gain = res.perf_metrics["holding_p_gain"]
    perf_status = "✅ PASS" if np.isclose(m_gain, e_gain, atol=1e-7) else "❌ FAIL"
    print(
        f"LAYER 3: PERFORMANCE | Holding Gain: {e_gain:.4f} vs Manual: {m_gain:.4f} | {perf_status}"
    )

    # --------------------------------------------------------------------------
    # DISPLAY TABLES
    # --------------------------------------------------------------------------
    print("=" * 85)
    display(
        sel_df.set_index("Rank")
        .style.format({"Engine": "{:.6f}", "Manual": "{:.6f}", "Delta": "{:.8f}"})
        .set_caption("Layer 2: Selection Reconciliation")
    )

    perf_summary = pd.DataFrame(
        {
            "Metric": ["Holding Gain", "Holding Sharpe", "Sharpe (ATRP)"],
            "Engine": [
                e_gain,
                res.perf_metrics["holding_p_sharpe"],
                res.perf_metrics["holding_p_sharpe_atrp"],
            ],
            "Manual": [m_gain, m_sharpe, m_s_atrp],
        }
    ).set_index("Metric")
    perf_summary["Delta"] = perf_summary["Engine"] - perf_summary["Manual"]
    display(
        perf_summary.style.format("{:.8f}").set_caption(
            "Layer 3: Performance Reconciliation"
        )
    )


def verify_engine_results_long_form(
    audit_pack, engine
):  # <--- FIXED: Added engine parameter
    """
    Run independent verification of Engine run results
    """
    # 1. SETUP
    res = audit_pack[0]

    print("=" * 78)
    print("MODULAR EVENT-DRIVEN AUDIT -- START")
    print("=" * 78)

    if res.debug_data is None:
        raise ValueError("❌ Audit failed: The Engine was run with debug=False.")

    m = res.perf_metrics
    init_weights = res.initial_weights
    periods = {
        "Full": (res.start_date, res.holding_end_date),
        "Lookback": (res.start_date, res.decision_date),
        "Holding": (res.buy_date, res.holding_end_date),
    }

    # 2. FETCH RAW DATA
    p_raw_components = res.debug_data["portfolio_raw_components"]
    b_raw_components = res.debug_data["benchmark_raw_components"]
    p_ohlcv = p_raw_components["ohlcv_raw"]
    p_raw_price = p_raw_components["prices"]
    b_ohlcv = b_raw_components["ohlcv_raw"]
    b_raw_price = b_raw_components["prices"]

    # VOLATILITY MATH KERNEL
    def calculate_manual_vol(df_ohlcv, atr_period=14):
        df = df_ohlcv.copy()
        df["PC"] = df.groupby(level="Ticker")["Adj Close"].shift(1)
        tr = pd.concat(
            [
                df["Adj High"] - df["Adj Low"],
                (df["Adj High"] - df["PC"]).abs(),
                (df["Adj Low"] - df["PC"]).abs(),
            ],
            axis=1,
        ).max(axis=1)
        atr = (
            tr.groupby(level="Ticker")
            .ewm(alpha=1 / atr_period, adjust=False)
            .mean()
            .reset_index(level=0, drop=True)
        )
        return (atr / df["Adj Close"]).unstack(level="Ticker"), (
            tr / df["Adj Close"]
        ).unstack(level="Ticker")

    _, p_manual_trp = calculate_manual_vol(p_ohlcv)
    _, b_manual_trp = calculate_manual_vol(b_ohlcv)

    p_raw = {
        "price": p_raw_price,
        "atrp": p_raw_components["atrp"],
        "trp": p_manual_trp,
    }
    b_raw = {
        "price": b_raw_price,
        "atrp": b_raw_components["atrp"],
        "trp": b_manual_trp,
    }

    # AUDIT KERNEL
    def run_period_audit(df_p, df_atrp, df_trp, weights):
        norm_prices = df_p.div(df_p.bfill().iloc[0])
        weighted_components = norm_prices.mul(weights, axis=1)
        equity_curve = weighted_components.sum(axis=1)
        current_weights = weighted_components.div(equity_curve, axis=0)
        port_atrp = (current_weights * df_atrp).sum(axis=1)
        port_trp = (current_weights * df_trp).sum(axis=1)
        returns = equity_curve.pct_change()
        v_mask = returns.notna()
        gain = (equity_curve.iloc[-1] / equity_curve.iloc[0]) - 1
        sharpe = (
            (returns.mean() / returns.std()) * np.sqrt(252) if returns.std() > 0 else 0
        )
        s_atrp = returns.mean() / port_atrp.where(v_mask).mean()
        s_trp = returns.mean() / port_trp.where(v_mask).mean()
        return gain, sharpe, s_atrp, s_trp

    audit_rows = []
    for p_label, (d_start, d_end) in periods.items():
        g, s, sa, st = run_period_audit(
            p_raw["price"].loc[d_start:d_end],
            p_raw["atrp"].loc[d_start:d_end],
            p_raw["trp"].loc[d_start:d_end],
            init_weights,
        )
        b_ticker = b_raw["price"].columns[0]
        bg, bs, bsa, bst = run_period_audit(
            b_raw["price"].loc[d_start:d_end],
            b_raw["atrp"].loc[d_start:d_end],
            b_raw["trp"].loc[d_start:d_end],
            pd.Series({b_ticker: 1.0}),
        )

        mapping = [
            ("Gain", g, f"{p_label.lower()}_p_gain", bg, f"{p_label.lower()}_b_gain"),
            (
                "Sharpe",
                s,
                f"{p_label.lower()}_p_sharpe",
                bs,
                f"{p_label.lower()}_b_sharpe",
            ),
            (
                "Sharpe (ATRP)",
                sa,
                f"{p_label.lower()}_p_sharpe_atrp",
                bsa,
                f"{p_label.lower()}_b_sharpe_atrp",
            ),
            (
                "Sharpe (TRP)",
                st,
                f"{p_label.lower()}_p_sharpe_trp",
                bst,
                f"{p_label.lower()}_b_sharpe_trp",
            ),
        ]

        for metric, manual_p, key_p, manual_b, key_b in mapping:
            audit_rows.append(
                {
                    "Entity": "Group",
                    "Period": p_label,
                    "Metric": metric,
                    "Engine": m.get(key_p, 0),
                    "Manual": manual_p,
                    "Delta": m.get(key_p, 0) - manual_p,
                }
            )
            audit_rows.append(
                {
                    "Entity": "Benchmark",
                    "Period": p_label,
                    "Metric": metric,
                    "Engine": m.get(key_b, 0),
                    "Manual": manual_b,
                    "Delta": m.get(key_b, 0) - manual_b,
                }
            )

    df_final = pd.DataFrame(audit_rows)
    df_final["Status"] = df_final["Delta"].apply(
        lambda x: "✅ PASS" if abs(x) < 1e-7 else "❌ FAIL"
    )
    print(f"📝 FINAL INTEGRITY REPORT (MODULAR)")
    display(
        df_final.pivot_table(
            index=["Entity", "Metric"],
            columns="Period",
            values="Status",
            aggfunc="first",
        )
    )
    display(
        df_final.style.format(
            {"Engine": "{:.8f}", "Manual": "{:.8f}", "Delta": "{:.8f}"}
        )
    )

    # ==============================================================================
    # SURVIVAL AUDIT
    # ==============================================================================
    print("\n" + "=" * 78)
    print("SURVIVAL AUDIT -- START")
    audit = res.debug_data["audit_liquidity"]
    snapshot = audit["universe_snapshot"]
    thresholds = res.debug_data.get("inputs_snapshot").quality_thresholds

    manual_quantile_val = snapshot["RollMedDollarVol"].quantile(
        thresholds["min_liquidity_percentile"]
    )
    manual_final_cutoff = max(
        manual_quantile_val, thresholds["min_median_dollar_volume"]
    )

    manual_survivors = snapshot[
        (snapshot["RollMedDollarVol"] >= manual_final_cutoff)
        & (snapshot["RollingStalePct"] <= thresholds["max_stale_pct"])
        & (snapshot["RollingSameVolCount"] <= thresholds["max_same_vol_count"])
    ]

    s_match = (
        "✅ PASS" if audit["tickers_passed"] == len(manual_survivors) else "❌ FAIL"
    )
    print(
        f"Survival Integrity: {s_match} (Engine: {audit['tickers_passed']} vs Manual: {len(manual_survivors)})"
    )

    # ==============================================================================
    # UNIVERSAL SELECTION AUDIT (The logic that previously failed)
    # ==============================================================================
    print("\n" + "=" * 78)
    print("UNIVERSAL SELECTION AUDIT -- START")

    debug = res.debug_data
    inputs = debug["inputs_snapshot"]
    survivors = snapshot.query("Passed_Final").index.tolist()
    idx = pd.IndexSlice

    # USES THE PASSED 'engine' INSTANCE
    feat_period = engine.features_df.loc[
        idx[survivors, res.start_date : res.decision_date], :
    ]
    atrp_lookback_mean = feat_period["ATRP"].groupby(level="Ticker").mean()
    trp_lookback_mean = feat_period["TRP"].groupby(level="Ticker").mean()
    feat_current = engine.features_df.xs(res.decision_date, level="Date").reindex(
        survivors
    )
    lookback_prices = engine.df_close.loc[res.start_date : res.decision_date, survivors]

    audit_obs: MarketObservation = {
        "lookback_close": lookback_prices,
        "lookback_returns": lookback_prices.ffill().pct_change(),
        "atrp": atrp_lookback_mean,
        "trp": trp_lookback_mean,
        "rsi": feat_current["RSI"],
        "rel_strength": feat_current["RelStrength"],
        "vol_regime": feat_current["VolRegime"],
        "rvol": feat_current["RVol"],
        "spy_rvol": feat_current["Spy_RVol"],
        "obv_score": feat_current["OBV_Score"],
        "spy_obv_score": feat_current["Spy_OBV_Score"],
        "roc_1": feat_current["ROC_1"],
        "roc_3": feat_current["ROC_3"],
        "roc_5": feat_current["ROC_5"],
        "roc_10": feat_current["ROC_10"],
        "roc_21": feat_current["ROC_21"],
    }

    manual_scores = METRIC_REGISTRY[inputs.metric](audit_obs)
    audit_results = []
    for rank, ticker in enumerate(res.tickers, start=inputs.rank_start):
        eng_score = res.results_df.loc[ticker, "Strategy Value"]
        aud_score = manual_scores.loc[ticker]
        audit_results.append(
            {
                "Rank": rank,
                "Ticker": ticker,
                "Engine_Score": eng_score,
                "Audit_Score": aud_score,
                "Delta": eng_score - aud_score,
                "Status": "✅ PASS" if np.isclose(eng_score, aud_score) else "❌ FAIL",
            }
        )

    display(
        pd.DataFrame(audit_results)
        .set_index("Rank")
        .style.format("{:.8f}", subset=["Engine_Score", "Audit_Score", "Delta"])
    )
    print("=" * 78)


#

In [3]:
# ==============================================================================
# SECTION H: UTILITIES
# ==============================================================================


def export_debug_to_csv(audit_pack, source_label="Audit"):
    """
    High-Transparency Exporter (Hardened Version).
    Dumps the entire simulation state into a folder for manual Excel verification.
    """
    if not audit_pack or not audit_pack[0]:
        print("❌ Error: Audit Pack is empty. Run a simulation first.")
        return

    data = audit_pack[0]
    # Handle the fact that 'inputs' might be a key or a dataclass attribute
    inputs = data.get("inputs")

    # 1. Folder Setup
    date_str = inputs.start_date.strftime("%Y-%m-%d")
    strat = inputs.metric.replace(" ", "").replace("(", "").replace(")", "")
    folder_name = f"{source_label}_{strat}_{date_str}"

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    print(f"📂 [AUDIT EXPORT] Folder: ./{folder_name}/")

    def process_item(item, path_prefix=""):
        # A. Handle Nested Dicts
        if isinstance(item, dict):
            for k, v in item.items():
                process_item(v, f"{path_prefix}{k}_" if path_prefix else f"{k}_")

        # B. Handle DataFrames (Matrices - High Precision)
        elif isinstance(item, pd.DataFrame):
            fn = f"Matrix_{path_prefix.strip('_')}.csv"
            item.to_csv(os.path.join(folder_name, fn), float_format="%.8f")
            print(f"   ✅ Matrix: {fn}")

        # C. Handle Series (Vectors)
        elif isinstance(item, pd.Series):
            fn = f"Vector_{path_prefix.strip('_')}.csv"
            item.to_frame().to_csv(os.path.join(folder_name, fn), float_format="%.8f")
            print(f"   ✅ Vector: {fn}")

        # D. Handle Dataclasses (Metadata & Results)
        elif is_dataclass(item):
            class_name = item.__class__.__name__
            fn = f"Summary_{class_name}_{path_prefix.strip('_')}".strip("_") + ".csv"

            # --- THE FIX: Create a Safe Dictionary for Pandas ---
            raw_dict = asdict(item)
            summary_ready_dict = {}

            for k, v in raw_dict.items():
                # If it's a big data object, just note its existence in the summary
                if isinstance(v, (pd.DataFrame, pd.Series)):
                    summary_ready_dict[k] = f"<{v.__class__.__name__} shape={v.shape}>"
                # If it's a list or dict (the crash cause), stringify it for Excel
                elif isinstance(v, (list, dict)):
                    summary_ready_dict[k] = str(v)
                else:
                    summary_ready_dict[k] = v

            # Save the clean key-value summary
            pd.DataFrame.from_dict(
                summary_ready_dict, orient="index", columns=["Value"]
            ).to_csv(os.path.join(folder_name, fn))
            print(f"   📑 Summary: {fn}")

            # E. RECURSION: Now find the actual DataFrames inside the dataclass
            # We iterate the object attributes directly to avoid the 'asdict' list confusion
            for k in item.__dataclass_fields__.keys():
                val = getattr(item, k)
                if isinstance(val, (pd.DataFrame, pd.Series, dict)):
                    process_item(val, f"{path_prefix}{k}_")

    # 3. Execute Extraction
    process_item(data)
    print(f"\n✨ Export Complete. Open ./{folder_name}/ to verify results.")


def export_audit_to_excel(audit_pack, filename="Audit_Verification_Report.xlsx"):
    """
    Consolidates the audit_pack into a multi-sheet Excel workbook.
    Organizes data by shared axes (Date vs Ticker) for manual formula checking.
    """
    if not audit_pack or not audit_pack[0]:
        print("❌ Error: Audit Pack is empty.")
        return

    data = audit_pack[0]
    res = data["results"]
    inputs = data["inputs"]
    debug = data.get("debug", {})

    print(f"📂 [EXCEL AUDIT] Creating Report: {filename}")

    with pd.ExcelWriter(filename, engine="openpyxl") as writer:

        # --- SHEET 1: OVERVIEW (The Settings & Final Totals) ---
        # Combines Input settings and Result scalars into one vertical table
        meta_dict = {
            **asdict(inputs),
            **{
                k: v
                for k, v in asdict(res).items()
                if not isinstance(v, (pd.DataFrame, pd.Series, dict))
            },
        }
        # Stringify lists/dicts to prevent Excel/Pandas export crashes
        clean_meta = {
            k: (str(v) if isinstance(v, (list, dict)) else v)
            for k, v in meta_dict.items()
        }

        df_overview = pd.DataFrame.from_dict(
            clean_meta, orient="index", columns=["Value"]
        )
        df_overview.to_excel(writer, sheet_name="OVERVIEW")

        # --- SHEET 2: DAILY_AUDIT (Axis = Date) ---
        # Concatenates everything that happens day-by-day
        daily_items = {
            "Port_Value": res.portfolio_series,
            "Port_Ret": QuantUtils.compute_returns(res.portfolio_series),
            "Port_ATRP": res.portfolio_atrp_series,  # <--- DIRECT ACCESS
            "Port_TRP": res.portfolio_trp_series,  # <--- DIRECT ACCESS
            "Bench_Value": res.benchmark_series,
            "Bench_Ret": QuantUtils.compute_returns(res.benchmark_series),
            "Bench_ATRP": res.benchmark_atrp_series,  # <--- DIRECT ACCESS
            "Bench_TRP": res.benchmark_trp_series,  # <--- DIRECT ACCESS
        }

        # Filter out None values and concatenate side-by-side
        df_daily = pd.concat(
            {k: v for k, v in daily_items.items() if v is not None}, axis=1
        )
        df_daily.to_excel(writer, sheet_name="DAILY_AUDIT", float_format="%.8f")

        # --- SHEET 3: SELECTION_SNAPSHOT (Axis = Ticker) ---
        # Focuses on the selected 10-20 tickers and their performance
        if "full_universe_ranking" in debug:
            df_rank = debug["full_universe_ranking"]
            # Filter the leaderboard for only the tickers we actually bought
            df_composition = df_rank.reindex(res.tickers)
            df_composition.to_excel(
                writer, sheet_name="PORTFOLIO_SNAPSHOT", float_format="%.8f"
            )

        # --- SHEET 4: FULL_UNIVERSE_RANKING ---
        if "full_universe_ranking" in debug:
            debug["full_universe_ranking"].to_excel(
                writer, sheet_name="FULL_RANKING", float_format="%.8f"
            )

        # --- SHEET 5: RAW_PRICES_MATRIX ---
        if "portfolio_raw_components" in debug:
            raw_p = debug["portfolio_raw_components"].get("prices")
            if raw_p is not None:
                raw_p.to_excel(writer, sheet_name="RAW_PRICES", float_format="%.8f")

        # --- SHEET 6: RAW_VOL_MATRIX (TRP) ---
        if "portfolio_raw_components" in debug:
            # Re-extracting TRP matrix for the specific tickers
            raw_v = debug["portfolio_raw_components"].get(
                "atrp"
            )  # Or trp if stored specifically
            if raw_v is not None:
                raw_v.to_excel(writer, sheet_name="RAW_VOL_DATA", float_format="%.8f")

    print(f"✨ Audit Report Complete. Manual verification ready in {filename}")


def print_nested(d, indent=0, width=4):
    """Pretty-print nested containers.
    Leaves are rendered as two lines:  key\\nvalue ."""
    spacing = " " * indent

    def _kind(node):
        if not isinstance(node, dict):
            return None
        return "sep" if all(isinstance(v, dict) for v in node.values()) else "nest"

    if isinstance(d, dict):
        for k, v in d.items():
            kind = _kind(v)
            tag = "" if kind is None else f"  [{'SEP' if kind == 'sep' else 'NEST'}]"
            print(f"{spacing}{k}{tag}")
            print_nested(v, indent + width, width)

    elif isinstance(d, (list, tuple)):
        for idx, item in enumerate(d):
            print(f"{spacing}[{idx}]")
            print_nested(item, indent + width, width)

    else:  # leaf – primitive value
        print(f"{spacing}{d}")


def get_ticker_OHLCV(
    df_ohlcv: pd.DataFrame,
    tickers: Union[str, List[str]],
    date_start: str,
    date_end: str,
    return_format: str = "dataframe",
    verbose: bool = True,
) -> Union[pd.DataFrame, dict]:
    """
    Get OHLCV data for specified tickers within a date range.

    Parameters
    ----------
    df_ohlcv : pd.DataFrame
        DataFrame with MultiIndex of (ticker, date) and OHLCV columns
    tickers : str or list of str
        Ticker symbol(s) to retrieve
    date_start : str
        Start date in 'YYYY-MM-DD' format
    date_end : str
        End date in 'YYYY-MM-DD' format
    return_format : str, optional
        Format to return data in. Options:
        - 'dataframe': Single DataFrame with MultiIndex (default)
        - 'dict': Dictionary with tickers as keys and DataFrames as values
        - 'separate': List of separate DataFrames for each ticker
    verbose : bool, optional
        Whether to print summary information (default: True)

    Returns
    -------
    Union[pd.DataFrame, dict, list]
        Filtered OHLCV data in specified format

    Raises
    ------
    ValueError
        If input parameters are invalid
    KeyError
        If tickers not found in DataFrame

    Examples
    --------
    >>> # Get data for single ticker
    >>> vlo_data = get_ticker_OHLCV(df_ohlcv, 'VLO', '2025-08-13', '2025-09-04')

    >>> # Get data for multiple tickers
    >>> multi_data = get_ticker_OHLCV(df_ohlcv, ['VLO', 'JPST'], '2025-08-13', '2025-09-04')

    >>> # Get data as dictionary
    >>> data_dict = get_ticker_OHLCV(df_ohlcv, ['VLO', 'JPST'], '2025-08-13',
    ...                              '2025-09-04', return_format='dict')
    """

    # Input validation
    if not isinstance(df_ohlcv, pd.DataFrame):
        raise TypeError("df_ohlcv must be a pandas DataFrame")

    if not isinstance(df_ohlcv.index, pd.MultiIndex):
        raise ValueError("DataFrame must have MultiIndex of (ticker, date)")

    if len(df_ohlcv.index.levels) != 2:
        raise ValueError("MultiIndex must have exactly 2 levels: (ticker, date)")

    # Convert single ticker to list for consistent processing
    if isinstance(tickers, str):
        tickers = [tickers]
    elif not isinstance(tickers, list):
        raise TypeError("tickers must be a string or list of strings")

    # Convert dates to Timestamps
    try:
        start_date = pd.Timestamp(date_start)
        end_date = pd.Timestamp(date_end)
    except ValueError as e:
        raise ValueError(f"Invalid date format. Use 'YYYY-MM-DD': {e}")

    if start_date > end_date:
        raise ValueError("date_start must be before or equal to date_end")

    # Check if tickers exist in the DataFrame
    available_tickers = df_ohlcv.index.get_level_values(0).unique()
    missing_tickers = [t for t in tickers if t not in available_tickers]

    if missing_tickers:
        raise KeyError(f"Ticker(s) not found in DataFrame: {missing_tickers}")

    # Filter the data using MultiIndex slicing
    try:
        filtered_data = df_ohlcv.loc[(tickers, slice(date_start, date_end)), :]
    except Exception as e:
        raise ValueError(f"Error filtering data: {e}")

    # Handle empty results
    if filtered_data.empty:
        if verbose:
            print(
                f"No data found for tickers {tickers} in date range {date_start} to {date_end}"
            )
        return filtered_data

    # Print summary if verbose
    if verbose:
        print(
            f"Data retrieved for {len(tickers)} ticker(s) from {date_start} to {date_end}"
        )
        print(f"Total rows: {len(filtered_data)}")
        print(
            f"Date range in data: {filtered_data.index.get_level_values(1).min()} to "
            f"{filtered_data.index.get_level_values(1).max()}"
        )

        # Print ticker-specific counts
        ticker_counts = filtered_data.index.get_level_values(0).value_counts()
        for ticker in tickers:
            count = ticker_counts.get(ticker, 0)
            if count > 0:
                print(f"  {ticker}: {count} rows")
            else:
                print(f"  {ticker}: No data in range")

    # Return in requested format
    if return_format == "dict":
        result = {}
        for ticker in tickers:
            try:
                result[ticker] = filtered_data.xs(ticker, level=0).loc[
                    date_start:date_end
                ]
            except KeyError:
                result[ticker] = pd.DataFrame()
        return result

    elif return_format == "separate":
        result = []
        for ticker in tickers:
            try:
                result.append(
                    filtered_data.xs(ticker, level=0).loc[date_start:date_end]
                )
            except KeyError:
                result.append(pd.DataFrame())
        return result

    elif return_format == "dataframe":
        return filtered_data

    else:
        raise ValueError(
            f"Invalid return_format: {return_format}. "
            f"Must be 'dataframe', 'dict', or 'separate'"
        )


def get_ticker_features(
    features_df: pd.DataFrame,
    tickers: Union[str, List[str]],
    date_start: str,
    date_end: str,
    return_format: str = "dataframe",
    verbose: bool = True,
) -> Union[pd.DataFrame, dict]:
    """
    Get features data for specified tickers within a date range.

    Parameters
    ----------
    features_df : pd.DataFrame
        DataFrame with MultiIndex of (ticker, date) and feature columns
    tickers : str or list of str
        Ticker symbol(s) to retrieve
    date_start : str
        Start date in 'YYYY-MM-DD' format
    date_end : str
        End date in 'YYYY-MM-DD' format
    return_format : str, optional
        Format to return data in. Options:
        - 'dataframe': Single DataFrame with MultiIndex (default)
        - 'dict': Dictionary with tickers as keys and DataFrames as values
        - 'separate': List of separate DataFrames for each ticker
    verbose : bool, optional
        Whether to print summary information (default: True)

    Returns
    -------
    Union[pd.DataFrame, dict, list]
        Filtered features data in specified format
    """
    # Convert single ticker to list for consistent processing
    if isinstance(tickers, str):
        tickers = [tickers]

    # Filter the data using MultiIndex slicing
    try:
        filtered_data = features_df.loc[(tickers, slice(date_start, date_end)), :]
    except Exception as e:
        if verbose:
            print(f"Error filtering data: {e}")
        return pd.DataFrame() if return_format == "dataframe" else {}

    # Handle empty results
    if filtered_data.empty:
        if verbose:
            print(
                f"No data found for tickers {tickers} in date range {date_start} to {date_end}"
            )
        return filtered_data

    # Print summary if verbose
    if verbose:
        print(
            f"Features data retrieved for {len(tickers)} ticker(s) from {date_start} to {date_end}"
        )
        print(f"Total rows: {len(filtered_data)}")
        print(
            f"Date range in data: {filtered_data.index.get_level_values(1).min()} to "
            f"{filtered_data.index.get_level_values(1).max()}"
        )
        print(f"Available features: {', '.join(filtered_data.columns.tolist())}")

        # Print ticker-specific counts
        ticker_counts = filtered_data.index.get_level_values(0).value_counts()
        for ticker in tickers:
            count = ticker_counts.get(ticker, 0)
            if count > 0:
                print(f"  {ticker}: {count} rows")
            else:
                print(f"  {ticker}: No data in range")

    # Return in requested format
    if return_format == "dict":
        result = {}
        for ticker in tickers:
            try:
                result[ticker] = filtered_data.xs(ticker, level=0).loc[
                    date_start:date_end
                ]
            except KeyError:
                result[ticker] = pd.DataFrame()
        return result

    elif return_format == "separate":
        result = []
        for ticker in tickers:
            try:
                result.append(
                    filtered_data.xs(ticker, level=0).loc[date_start:date_end]
                )
            except KeyError:
                result.append(pd.DataFrame())
        return result

    elif return_format == "dataframe":
        return filtered_data

    else:
        raise ValueError(
            f"Invalid return_format: {return_format}. "
            f"Must be 'dataframe', 'dict', or 'separate'"
        )


def create_combined_dict(
    df_ohlcv: pd.DataFrame,
    features_df: pd.DataFrame,
    tickers: Union[str, List[str]],
    date_start: str,
    date_end: str,
    verbose: bool = True,
) -> dict:
    """
    Create a combined dictionary with both OHLCV and features data for each ticker.

    Parameters:
    -----------
    df_ohlcv : pd.DataFrame
        DataFrame with OHLCV data (MultiIndex: ticker, date)
    features_df : pd.DataFrame
        DataFrame with features data (MultiIndex: ticker, date)
    tickers : str or list of str
        Ticker symbol(s) to retrieve
    date_start : str
        Start date in 'YYYY-MM-DD' format
    date_end : str
        End date in 'YYYY-MM-DD' format
    verbose : bool, optional
        Whether to print progress information (default: True)

    Returns:
    --------
    dict
        Dictionary with tickers as keys and combined DataFrames (OHLCV + features) as values
    """
    # Convert single ticker to list
    if isinstance(tickers, str):
        tickers = [tickers]

    if verbose:
        print(f"Creating combined dictionary for {len(tickers)} ticker(s)")
        print(f"Date range: {date_start} to {date_end}")
        print("=" * 60)

    # Get OHLCV data as dictionary
    ohlcv_dict = get_ticker_OHLCV(
        df_ohlcv, tickers, date_start, date_end, return_format="dict", verbose=verbose
    )

    # Get features data as dictionary
    features_dict = get_ticker_features(
        features_df,
        tickers,
        date_start,
        date_end,
        return_format="dict",
        verbose=verbose,
    )

    # Create combined_dict
    combined_dict = {}

    for ticker in tickers:
        if verbose:
            print(f"\nProcessing {ticker}...")

        # Check if ticker exists in both dictionaries
        if ticker in ohlcv_dict and ticker in features_dict:
            ohlcv_data = ohlcv_dict[ticker]
            features_data = features_dict[ticker]

            # Check if both dataframes have data
            if not ohlcv_data.empty and not features_data.empty:
                # Combine OHLCV and features data
                # Note: Both dataframes have the same index (dates), so we can concatenate
                combined_df = pd.concat([ohlcv_data, features_data], axis=1)

                # Ensure proper index naming
                combined_df.index.name = "Date"

                # Store in combined_dict
                combined_dict[ticker] = combined_df

                if verbose:
                    print(f"  ✓ Successfully combined data")
                    print(f"  OHLCV shape: {ohlcv_data.shape}")
                    print(f"  Features shape: {features_data.shape}")
                    print(f"  Combined shape: {combined_df.shape}")
                    print(
                        f"  Date range: {combined_df.index.min()} to {combined_df.index.max()}"
                    )
            else:
                if verbose:
                    print(f"  ✗ Cannot combine: One or both dataframes are empty")
                    print(f"    OHLCV empty: {ohlcv_data.empty}")
                    print(f"    Features empty: {features_data.empty}")
                combined_dict[ticker] = pd.DataFrame()
        else:
            if verbose:
                print(f"  ✗ Ticker not found in both dictionaries")
                if ticker not in ohlcv_dict:
                    print(f"    Not in OHLCV data")
                if ticker not in features_dict:
                    print(f"    Not in features data")
            combined_dict[ticker] = pd.DataFrame()

    # Print summary
    if verbose:
        print("\n" + "=" * 60)
        print("SUMMARY")
        print("=" * 60)
        print(f"Total tickers processed: {len(tickers)}")

        tickers_with_data = [
            ticker for ticker, df in combined_dict.items() if not df.empty
        ]
        print(f"Tickers with combined data: {len(tickers_with_data)}")

        if tickers_with_data:
            print("\nTicker details:")
            for ticker in tickers_with_data:
                df = combined_dict[ticker]
                print(f"  {ticker}: {df.shape} - {df.index.min()} to {df.index.max()}")
                print(f"    Columns: {len(df.columns)}")

        empty_tickers = [ticker for ticker, df in combined_dict.items() if df.empty]
        if empty_tickers:
            print(f"\nTickers with no data: {', '.join(empty_tickers)}")

    return combined_dict


#

In [4]:
data_path = r"c:\Users\ping\Files_win10\python\py311\stocks\data\df_indices.parquet"

df_indices = pd.read_parquet(data_path, engine="pyarrow")

In [5]:
df_indices.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 143906 entries, ('^AXJO', Timestamp('1992-11-22 00:00:00')) to ('^VIX3M', Timestamp('2026-01-20 00:00:00'))
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Open   143906 non-null  float64
 1   Adj High   143906 non-null  float64
 2   Adj Low    143906 non-null  float64
 3   Adj Close  143906 non-null  float64
 4   Volume     143906 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 6.5+ MB


In [6]:
data_path = (
    r"c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet"
)

df_ohlcv = pd.read_parquet(data_path, engine="pyarrow")

In [7]:
df_ohlcv.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9478795 entries, ('A', Timestamp('1999-11-18 00:00:00')) to ('ZWS', Timestamp('2026-01-20 00:00:00'))
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Adj Open   float64
 1   Adj High   float64
 2   Adj Low    float64
 3   Adj Close  float64
 4   Volume     int64  
dtypes: float64(4), int64(1)
memory usage: 398.4+ MB


### DEBUG: Subset df_ohlcv to 11 tickers

In [8]:
# _tickers = ["A", "B", "C", "D", "E", "F", "G", "H", "IAU", "J"] + ["SPY"]

# # Subset ohlcv to 11 tickers
# print(f"Subset df_ohlcv to 11 tickers")
# df_ohlcv = df_ohlcv.loc[_tickers].sort_index()


# ohlcv_tickers = df_ohlcv.index.get_level_values(0).unique()
# print(f"ohlcv_tickers count: {len(ohlcv_tickers)}:\n{ohlcv_tickers}\n")

# print(df_ohlcv)

In [9]:
# ==============================================================================
# DATA PRE-COMPUTATION (The "Fast-Track" Setup)
# ==============================================================================

print("Calculating features... this might take about 3 minutes...")
features_df = generate_features(
    df_ohlcv=df_ohlcv,
    # atr_period=GLOBAL_SETTINGS["atr_period"],
    # quality_window=GLOBAL_SETTINGS["quality_window"],
    # quality_min_periods=GLOBAL_SETTINGS["quality_min_periods"],
)

print("🚀 Generating Wide Matrices for Instant Backtesting...")

# 1. Price Matrix
df_close_wide = df_ohlcv["Adj Close"].unstack(level=0)

# 2. Volatility Matrices (Unstack and Align)
# Using reindex_like ensures Dates and Tickers match df_close_wide exactly
print("   - Unstacking ATRP...")
df_atrp_wide = features_df["ATRP"].unstack(level=0).reindex_like(df_close_wide)

print("   - Unstacking TRP...")
df_trp_wide = features_df["TRP"].unstack(level=0).reindex_like(df_close_wide)

# 3. Handle Data Gaps (Sanitize the Wide Matrices)
# This prevents NaN propagation during matrix multiplication
if GLOBAL_SETTINGS["handle_zeros_as_nan"]:
    df_close_wide = df_close_wide.replace(0, np.nan)

# Forward fill up to the limit, then fill remaining with the "Disaster Detection" value
df_close_wide = df_close_wide.ffill(limit=GLOBAL_SETTINGS["max_data_gap_ffill"])
df_close_wide = df_close_wide.fillna(GLOBAL_SETTINGS["nan_price_replacement"])

print(
    "✅ Pre-computation Complete. df_close_wide, df_atrp_wide, and df_trp_wide are ready."
)

Calculating features... this might take about 3 minutes...
⚡ Generating Features (Base: 21d, Ratio Clip: 10.0)...
🚀 Generating Wide Matrices for Instant Backtesting...
   - Unstacking ATRP...
   - Unstacking TRP...
✅ Pre-computation Complete. df_close_wide, df_atrp_wide, and df_trp_wide are ready.


In [10]:
# Create once, use everywhere
master_engine1 = AlphaEngine(
    df_ohlcv, features_df, df_close_wide, df_atrp_wide, df_trp_wide
)

--- ⚙️ Initializing AlphaEngine v2.2 (Sanitized) ---


### Apply First Filter, filters tickers from 11 to 7

In [11]:
# ==============================================================================
# STEP 1: RUN THE COARSE FILTER (THE NET)
# ==============================================================================
# Returns the audit_pack and the engine instance (which we capture as _)
audit1, engine1 = plot_walk_forward_analyzer(
    engine=master_engine1,  # Persistent Engine
    default_start_date="2025-12-10",  # Shared Decision Date
    default_lookback=252,  # Long-term stability check
    default_holding=5,  # Holding period for metrics
    default_strategy="Sharpe (ATRP)",  # The "Quality" Net
    default_rank_end=100,  # Keep the top 100 candidates
    quality_thresholds=GLOBAL_SETTINGS["thresholds"],
    debug=True,  # Enable debug_data for Layer 1-3 Audit
)

FigureWidget({
    'data': [{'line': {'width': 2},
              'name': 'RY',
              'type': 'scatter',
              'uid': 'fa7b1aaf-bbd3-4ce8-a37a-2e22228725db',
              'visible': True,
              'x': array([datetime.datetime(2024, 12, 6, 0, 0),
                          datetime.datetime(2024, 12, 9, 0, 0),
                          datetime.datetime(2024, 12, 10, 0, 0), ...,
                          datetime.datetime(2025, 12, 16, 0, 0),
                          datetime.datetime(2025, 12, 17, 0, 0),
                          datetime.datetime(2025, 12, 18, 0, 0)], dtype=object),
              'y': array([1.        , 0.99929161, 0.99897405, ..., 1.35466115, 1.34586729,
                          1.36093085])},
             {'line': {'width': 2},
              'name': 'KLAC',
              'type': 'scatter',
              'uid': '0f7eb302-9ffc-4a38-9b43-14643b66cacc',
              'visible': True,
              'x': array([datetime.datetime(2024, 12, 6, 0, 0),


In [12]:
res1 = visualize_audit_structure(audit1)

🔍 HIGH-TRANSPARENCY AUDIT MAP
[  0] 📂 audit_pack (len=1)
[  1]   📦 index_0 (EngineOutput)
[  2]     📈 portfolio_series (shape=(259,))
[  3]     📈 benchmark_series (shape=(259,))
[  4]     🧮 normalized_plot_data (shape=(259, 100))
[  5]     📂 tickers (len=100)
[  6]       📄 index_0 (str)
[  7]       📄 index_1 (str)
[  8]       📄 index_2 (str)
[  9]       📄 index_3 (str)
[ 10]       📄 index_4 (str)
[ 11]       📄 index_5 (str)
[ 12]       📄 index_6 (str)
[ 13]       📄 index_7 (str)
[ 14]       📄 index_8 (str)
[ 15]       📄 index_9 (str)
[ 16]       📄 index_10 (str)
[ 17]       📄 index_11 (str)
[ 18]       📄 index_12 (str)
[ 19]       📄 index_13 (str)
[ 20]       📄 index_14 (str)
[ 21]       📄 index_15 (str)
[ 22]       📄 index_16 (str)
[ 23]       📄 index_17 (str)
[ 24]       📄 index_18 (str)
[ 25]       📄 index_19 (str)
[ 26]       📄 index_20 (str)
[ 27]       📄 index_21 (str)
[ 28]       📄 index_22 (str)
[ 29]       📄 index_23 (str)
[ 30]       📄 index_24 (str)
[ 31]       📄 index_25 (s

In [13]:
verify_engine_results_short_form(audit_pack=audit1, engine=master_engine1)

🕵️  STARTING MULTI-LAYER AUDIT: Sharpe (ATRP) @ 2025-12-10

*************************************************************************************
⚠️  ASSUMPTION: Verification logic is independent, but trusts Engine source DataFrames
   (engine.features_df, engine.df_close, and debug['portfolio_raw_components'])
*************************************************************************************


LAYER 1: SURVIVAL  | Universe: 1586 -> Survivors: 940 | ✅ PASS
LAYER 2: SELECTION | Strategy: Sharpe (ATRP) | Selection Match: ✅ PASS
LAYER 3: PERFORMANCE | Holding Gain: -0.0295 vs Manual: -0.0295 | ✅ PASS


,Ticker,Engine,Manual,Delta,Match
Rank,,,,,
1,SGOV,0.766401,0.766401,0.00000000,True
2,SHV,0.764396,0.764396,0.00000000,True
3,BIL,0.728330,0.728330,0.00000000,True
4,MINT,0.522852,0.522852,0.00000000,True
5,USFR,0.511571,0.511571,0.00000000,True
6,BOXX,0.507358,0.507358,0.00000000,True
7,PULS,0.401253,0.401253,0.00000000,True
8,JPST,0.319077,0.319077,0.00000000,True
9,TD,0.166750,0.166750,0.00000000,True


,Engine,Manual,Delta
Metric,,,
Holding Gain,-0.02945748,-0.02945748,0.00000000
Holding Sharpe,-9.07518695,-9.07518695,0.00000000
Sharpe (ATRP),-0.20510750,-0.20510750,0.00000000


In [14]:
verify_engine_results_long_form(audit_pack=audit1, engine=master_engine1)

MODULAR EVENT-DRIVEN AUDIT -- START
📝 FINAL INTEGRITY REPORT (MODULAR)


Period                     Full Holding Lookback
Entity    Metric                                
Benchmark Gain           ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe         ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe (ATRP)  ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe (TRP)   ✅ PASS  ✅ PASS   ✅ PASS
Group     Gain           ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe         ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe (ATRP)  ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe (TRP)   ✅ PASS  ✅ PASS   ✅ PASS

,Entity,Period,Metric,Engine,Manual,Delta,Status
0,Group,Full,Gain,0.90444402,0.90444402,0.00000000,✅ PASS
1,Benchmark,Full,Gain,0.12649397,0.12649397,0.00000000,✅ PASS
2,Group,Full,Sharpe,2.85804428,2.85804428,0.00000000,✅ PASS
3,Benchmark,Full,Sharpe,0.69170935,0.69170935,0.00000000,✅ PASS
4,Group,Full,Sharpe (ATRP),0.07950145,0.07950145,0.00000000,✅ PASS
5,Benchmark,Full,Sharpe (ATRP),0.04079627,0.04079627,0.00000000,✅ PASS
6,Group,Full,Sharpe (TRP),0.07525575,0.07525575,0.00000000,✅ PASS
7,Benchmark,Full,Sharpe (TRP),0.03975231,0.03975231,0.00000000,✅ PASS
8,Group,Lookback,Gain,0.96318752,0.96318752,0.00000000,✅ PASS
9,Benchmark,Lookback,Gain,0.14497779,0.14497779,0.00000000,✅ PASS



SURVIVAL AUDIT -- START
Survival Integrity: ✅ PASS (Engine: 940 vs Manual: 940)

UNIVERSAL SELECTION AUDIT -- START


,Ticker,Engine_Score,Audit_Score,Delta,Status
Rank,,,,,
1,SGOV,0.76640099,0.76640099,0.00000000,✅ PASS
2,SHV,0.76439641,0.76439641,0.00000000,✅ PASS
3,BIL,0.72832972,0.72832972,0.00000000,✅ PASS
4,MINT,0.52285240,0.52285240,0.00000000,✅ PASS
5,USFR,0.51157082,0.51157082,0.00000000,✅ PASS
6,BOXX,0.50735798,0.50735798,0.00000000,✅ PASS
7,PULS,0.40125251,0.40125251,0.00000000,✅ PASS
8,JPST,0.31907714,0.31907714,0.00000000,✅ PASS
9,TD,0.16675036,0.16675036,0.00000000,✅ PASS


### Apply Second Filter Using Only Tickers Passed 1st Filter (7 tickers total), Start with 7 tickers, select the top 10

In [29]:
selected_tickers_1st_filter = audit1[0].tickers
print(
    f"selected_tickers_1st_filter count {len(selected_tickers_1st_filter)}: {selected_tickers_1st_filter}"
)

selected_tickers_1st_filter count 100: ['SGOV', 'SHV', 'BIL', 'MINT', 'USFR', 'BOXX', 'PULS', 'JPST', 'TD', 'VGSH', 'SHY', 'GLDM', 'SGOL', 'IAU', 'GLD', 'AU', 'JAAA', 'SLV', 'EWY', 'PSLV', 'STX', 'WDC', 'BTI', 'B', 'GDX', 'KGC', 'GDXJ', 'VCSH', 'SRLN', 'BKLN', 'NEM', 'CIEN', 'EFV', 'BSV', 'IGSB', 'INSM', 'LITE', 'BCS', 'HL', 'RGTI', 'WBD', 'IAG', 'GLW', 'CLS', 'AEM', 'MU', 'AVGO', 'APH', 'LRCX', 'EZU', 'GOOGL', 'GOOG', 'CAH', 'BBIO', 'QBTS', 'PAAS', 'SPDW', 'VEA', 'ITUB', 'VEU', 'KTOS', 'HOOD', 'IXUS', 'VXUS', 'GH', 'FNV', 'SCHF', 'EEM', 'IEMG', 'BE', 'WPM', 'TEL', 'HII', 'JNJ', 'VGK', 'KLAC', 'RTX', 'RY', 'TPR', 'SPIB', 'AG', 'BABA', 'EFA', 'MP', 'IEFA', 'NXT', 'BK', 'BBD', 'IDXX', 'GE', 'FIX', 'SGI', 'OKLO', 'PLTR', 'WELL', 'RPRX', 'CAT', 'ATI', 'CHRW', 'GSK']


In [35]:
selected_tickers_1st_filter = audit1[0].tickers
print(
    f"selected_tickers_1st_filter count {len(selected_tickers_1st_filter)}: {selected_tickers_1st_filter}"
)

# # Concatenate list with SPY without running res1 = visualize_audit_structure(audit1)
# selected_tickers_1st_filter = selected_tickers_1st_filter + ["SPY"]
# # fastest, memory-light way to remove duplicates (i.e. SPY) while preserving the original order of the tickers.
# selected_tickers_1st_filter = list(dict.fromkeys(selected_tickers_1st_filter))
# print(f"selected_tickers_1st_filter (unique): {selected_tickers_1st_filter}")

df_ohlcv_subset = df_ohlcv.loc[selected_tickers_1st_filter].sort_index()
features_df_subset = features_df.loc[selected_tickers_1st_filter].sort_index()
df_close_wide_subset = df_close_wide[selected_tickers_1st_filter]
df_atrp_wide_subset = df_atrp_wide[selected_tickers_1st_filter]
df_trp_wide_subset = df_trp_wide[selected_tickers_1st_filter]

# df_ohlcv_subset = df_ohlcv_subset.sort_index()
master_engine2 = AlphaEngine(
    df_ohlcv_subset,
    features_df_subset,
    df_close_wide_subset,
    df_atrp_wide_subset,
    df_trp_wide_subset,
)

selected_tickers_1st_filter count 100: ['SGOV', 'SHV', 'BIL', 'MINT', 'USFR', 'BOXX', 'PULS', 'JPST', 'TD', 'VGSH', 'SHY', 'GLDM', 'SGOL', 'IAU', 'GLD', 'AU', 'JAAA', 'SLV', 'EWY', 'PSLV', 'STX', 'WDC', 'BTI', 'B', 'GDX', 'KGC', 'GDXJ', 'VCSH', 'SRLN', 'BKLN', 'NEM', 'CIEN', 'EFV', 'BSV', 'IGSB', 'INSM', 'LITE', 'BCS', 'HL', 'RGTI', 'WBD', 'IAG', 'GLW', 'CLS', 'AEM', 'MU', 'AVGO', 'APH', 'LRCX', 'EZU', 'GOOGL', 'GOOG', 'CAH', 'BBIO', 'QBTS', 'PAAS', 'SPDW', 'VEA', 'ITUB', 'VEU', 'KTOS', 'HOOD', 'IXUS', 'VXUS', 'GH', 'FNV', 'SCHF', 'EEM', 'IEMG', 'BE', 'WPM', 'TEL', 'HII', 'JNJ', 'VGK', 'KLAC', 'RTX', 'RY', 'TPR', 'SPIB', 'AG', 'BABA', 'EFA', 'MP', 'IEFA', 'NXT', 'BK', 'BBD', 'IDXX', 'GE', 'FIX', 'SGI', 'OKLO', 'PLTR', 'WELL', 'RPRX', 'CAT', 'ATI', 'CHRW', 'GSK']
--- ⚙️ Initializing AlphaEngine v2.2 (Sanitized) ---


In [36]:
# ==============================================================================
# STEP 2: RUN THE SNIPER FILTER (THE FINE TUNING)
# ==============================================================================
# We use the master_engine again, but restricted to the audit_1 subset.
audit2, engine2 = plot_walk_forward_analyzer(
    engine=master_engine2,
    # universe_subset=audit_1[0].tickers,  # <--- The 100 candidates from Run 1
    default_start_date=audit1[0].decision_date,  # <--- Perfectly sync'd date
    default_strategy="Sharpe (TRP)",  # Faster, high-conviction metric
    default_lookback=10,  # Short-term timing window
    default_rank_end=10,  # Final 10 tradeable tickers
    quality_thresholds={
        "min_median_dollar_volume": 0,
        "min_liquidity_percentile": 0,  # Set to 0, otherwise will filter out bottom 40%
        "max_stale_pct": 0.05,
        "max_same_vol_count": 10,
    },
    debug=True,  # Keep enabled for verification
)

FigureWidget({
    'data': [{'line': {'width': 2}, 'type': 'scatter', 'uid': 'b0cf3488-6078-4267-ba32-a4da94f64752', 'visible': False},
             {'line': {'width': 2}, 'type': 'scatter', 'uid': 'ae1fdf1f-87b3-4b63-9ba8-706eb30c314f', 'visible': False},
             {'line': {'width': 2}, 'type': 'scatter', 'uid': 'cce12d86-2fd1-48c3-84e5-c705a18c2e71', 'visible': False},
             {'line': {'width': 2}, 'type': 'scatter', 'uid': '95ec7083-ee31-421b-8276-ffd4739db89c', 'visible': False},
             {'line': {'width': 2}, 'type': 'scatter', 'uid': '02548f91-d8d2-4fd5-9c8f-6599176776c4', 'visible': False},
             {'line': {'width': 2}, 'type': 'scatter', 'uid': '3bca8b3d-888c-46df-b3b3-6493fe4cd30e', 'visible': False},
             {'line': {'width': 2}, 'type': 'scatter', 'uid': '4d29da4f-13a5-439b-b61c-3efca1383c92', 'visible': False},
             {'line': {'width': 2}, 'type': 'scatter', 'uid': '6e6b6441-8971-4f6d-bbd5-dbb4982f942a', 'visible': False},
             {'li

In [37]:
selected_tickers_2nd_filter = audit2[0].tickers
print(
    f"selected_tickers_1st_filter count {len(selected_tickers_1st_filter)}: {selected_tickers_1st_filter}"
)
print(
    f"selected_tickers_2nd_filter count {len(selected_tickers_2nd_filter)}: {selected_tickers_2nd_filter}"
)

only_in_selected_tickers_2nd_filter = list(
    set(selected_tickers_2nd_filter) - set(selected_tickers_1st_filter)
)

print(
    f"\nWe asked for 10 tickers but it only return all 7 tickers that passed the 1st filter"
)
print(f"This is the CORRECT behavior")
print(
    f"only_in_selected_tickers_2nd_filter count {len(only_in_selected_tickers_2nd_filter)}: {only_in_selected_tickers_2nd_filter}"
)

AttributeError: 'NoneType' object has no attribute 'tickers'

In [ ]:
res2 = visualize_audit_structure(audit2)

🔍 HIGH-TRANSPARENCY AUDIT MAP
[  0] 📂 audit_pack (len=1)
[  1]   📦 index_0 (EngineOutput)
[  2]     📈 portfolio_series (shape=(17,))
[  3]     📈 benchmark_series (shape=(17,))
[  4]     🧮 normalized_plot_data (shape=(17, 10))
[  5]     📂 tickers (len=10)
[  6]       📄 index_0 (str)
[  7]       📄 index_1 (str)
[  8]       📄 index_2 (str)
[  9]       📄 index_3 (str)
[ 10]       📄 index_4 (str)
[ 11]       📄 index_5 (str)
[ 12]       📄 index_6 (str)
[ 13]       📄 index_7 (str)
[ 14]       📄 index_8 (str)
[ 15]       📄 index_9 (str)
[ 16]     📈 initial_weights (shape=(10,))
[ 17]     📂 perf_metrics (len=24)
[ 18]       🔢 full_p_gain (float)
[ 19]       🔢 full_p_sharpe (float)
[ 20]       🔢 full_p_sharpe_atrp (float)
[ 21]       🔢 full_p_sharpe_trp (float)
[ 22]       🔢 lookback_p_gain (float)
[ 23]       🔢 lookback_p_sharpe (float)
[ 24]       🔢 lookback_p_sharpe_atrp (float)
[ 25]       🔢 lookback_p_sharpe_trp (float)
[ 26]       🔢 holding_p_gain (float)
[ 27]       🔢 holding_p_sharpe (fl

In [ ]:
verify_engine_results_short_form(audit_pack=audit2, engine=master_engine2)

🕵️  STARTING MULTI-LAYER AUDIT: Sharpe (TRP) @ 2025-12-10

*************************************************************************************
⚠️  ASSUMPTION: Verification logic is independent, but trusts Engine source DataFrames
   (engine.features_df, engine.df_close, and debug['portfolio_raw_components'])
*************************************************************************************


LAYER 1: SURVIVAL  | Universe: 101 -> Survivors: 101 | ✅ PASS
LAYER 2: SELECTION | Strategy: Sharpe (TRP) | Selection Match: ✅ PASS
LAYER 3: PERFORMANCE | Holding Gain: 0.0030 vs Manual: 0.0030 | ✅ PASS


,Ticker,Engine,Manual,Delta,Match
Rank,,,,,
1,SHV,0.789797,0.789797,0.00000000,True
2,SGOV,0.753769,0.753769,0.00000000,True
3,BIL,0.689552,0.689552,0.00000000,True
4,MINT,0.667676,0.667676,0.00000000,True
5,WBD,0.646641,0.646641,0.00000000,True
6,TD,0.616072,0.616072,0.00000000,True
7,SLV,0.571533,0.571533,0.00000000,True
8,PSLV,0.568033,0.568033,0.00000000,True
9,RY,0.548323,0.548323,0.00000000,True


,Engine,Manual,Delta
Metric,,,
Holding Gain,0.00300983,0.00300983,0.00000000
Holding Sharpe,1.42832352,1.42832352,0.00000000
Sharpe (ATRP),0.04480872,0.04480872,0.00000000


In [ ]:
verify_engine_results_long_form(audit_pack=audit2, engine=master_engine2)

MODULAR EVENT-DRIVEN AUDIT -- START
📝 FINAL INTEGRITY REPORT (MODULAR)


Period                     Full Holding Lookback
Entity    Metric                                
Benchmark Gain           ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe         ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe (ATRP)  ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe (TRP)   ✅ PASS  ✅ PASS   ✅ PASS
Group     Gain           ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe         ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe (ATRP)  ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe (TRP)   ✅ PASS  ✅ PASS   ✅ PASS

,Entity,Period,Metric,Engine,Manual,Delta,Status
0,Group,Full,Gain,0.11161471,0.11161471,0.00000000,✅ PASS
1,Benchmark,Full,Gain,0.00214849,0.00214849,0.00000000,✅ PASS
2,Group,Full,Sharpe,13.83163784,13.83163784,0.00000000,✅ PASS
3,Benchmark,Full,Sharpe,0.41857488,0.41857488,0.00000000,✅ PASS
4,Group,Full,Sharpe (ATRP),0.45257130,0.45257130,0.00000000,✅ PASS
5,Benchmark,Full,Sharpe (ATRP),0.01277000,0.01277000,0.00000000,✅ PASS
6,Group,Full,Sharpe (TRP),0.40530472,0.40530472,0.00000000,✅ PASS
7,Benchmark,Full,Sharpe (TRP),0.01679050,0.01679050,0.00000000,✅ PASS
8,Group,Lookback,Gain,0.09901836,0.09901836,0.00000000,✅ PASS
9,Benchmark,Lookback,Gain,0.01859201,0.01859201,0.00000000,✅ PASS



SURVIVAL AUDIT -- START
Survival Integrity: ✅ PASS (Engine: 101 vs Manual: 101)

UNIVERSAL SELECTION AUDIT -- START


,Ticker,Engine_Score,Audit_Score,Delta,Status
Rank,,,,,
1,SHV,0.78979667,0.78979667,0.00000000,✅ PASS
2,SGOV,0.75376887,0.75376887,0.00000000,✅ PASS
3,BIL,0.68955244,0.68955244,0.00000000,✅ PASS
4,MINT,0.66767634,0.66767634,0.00000000,✅ PASS
5,WBD,0.64664101,0.64664101,0.00000000,✅ PASS
6,TD,0.61607185,0.61607185,0.00000000,✅ PASS
7,SLV,0.57153263,0.57153263,0.00000000,✅ PASS
8,PSLV,0.56803272,0.56803272,0.00000000,✅ PASS
9,RY,0.54832349,0.54832349,0.00000000,✅ PASS


### From Tickers Passed 2nd Filter, Combine Tickers' OHLCV with their Features

In [ ]:
my_audit_pack = audit2
tickers = my_audit_pack[0].tickers
start_date = my_audit_pack[0].start_date
decision_date = my_audit_pack[0].decision_date
buy_date = my_audit_pack[0].buy_date
end_date = my_audit_pack[0].holding_end_date

print(f"tickers: {tickers}")
print(f"start_date: {start_date}")
print(f"decision_date: {decision_date}")
print(f"buy_date: {buy_date}")
print(f"end_date: {end_date}")

combined = create_combined_dict(
    df_ohlcv=df_ohlcv.copy(),
    features_df=features_df,
    tickers=tickers,
    date_start=start_date,
    date_end=end_date,
    verbose=True,
)

tickers: ['SHV', 'SGOV', 'BIL', 'MINT', 'WBD', 'TD', 'SLV', 'PSLV', 'RY', 'BCS']
start_date: 2025-11-25 00:00:00
decision_date: 2025-12-10 00:00:00
buy_date: 2025-12-11 00:00:00
end_date: 2025-12-18 00:00:00
Creating combined dictionary for 10 ticker(s)
Date range: 2025-11-25 00:00:00 to 2025-12-18 00:00:00
Data retrieved for 10 ticker(s) from 2025-11-25 00:00:00 to 2025-12-18 00:00:00
Total rows: 170
Date range in data: 2025-11-25 00:00:00 to 2025-12-18 00:00:00
  SHV: 17 rows
  SGOV: 17 rows
  BIL: 17 rows
  MINT: 17 rows
  WBD: 17 rows
  TD: 17 rows
  SLV: 17 rows
  PSLV: 17 rows
  RY: 17 rows
  BCS: 17 rows
Features data retrieved for 10 ticker(s) from 2025-11-25 00:00:00 to 2025-12-18 00:00:00
Total rows: 170
Date range in data: 2025-11-25 00:00:00 to 2025-12-18 00:00:00
Available features: ATR, ATRP, TRP, RSI, RelStrength, VolRegime, RVol, Spy_RVol, OBV_Score, Spy_OBV_Score, ROC_1, ROC_3, ROC_5, ROC_10, ROC_21, RollingStalePct, RollMedDollarVol, RollingSameVolCount
  SHV: 17 rows

In [ ]:
print(f"The first and last 3 rows of Combinded dataframe")
for ticker in tickers:
    with pd.option_context("display.float_format", "{:.8f}".format):
        print(f"{ticker}:\n{combined[ticker].head(3)}\n")
        print(f"{ticker}:\n{combined[ticker].tail(3)}\n")

The first and last 3 rows of Combinded dataframe
SHV:
               Adj Open     Adj High      Adj Low    Adj Close   Volume        ATR       ATRP        TRP         RSI  RelStrength  VolRegime       RVol   Spy_RVol  OBV_Score  Spy_OBV_Score      ROC_1      ROC_3      ROC_5     ROC_10     ROC_21  RollingStalePct   RollMedDollarVol  RollingSameVolCount
Date                                                                                                                                                                                                                                                                                                        
2025-11-25 109.69200000 109.69200000 109.68200000 109.69200000  1872908 0.01838054 0.00016757 0.00009116 97.60370667   0.01800817 1.08410964 0.75066458 0.92283578 1.61488049     0.51428359 0.00009117 0.00053816 0.00072072 0.00145162 0.00282494       0.00000000 331112280.86399996           0.00000000
2025-11-26 109.71200000 109.72200000 109.71

In [ ]:
##########################################
##########################################

### USING universe_subset for 2nd Filter gives *WRONG* result. The result has ticker 'A' that is *NOT* in the Original 7 tickers    

### Apply Second Filter Using universe_subset (7 tickers total), Start with 7 tickers, select the top 10

In [ ]:
# ==============================================================================
# STEP 2: RUN THE SNIPER FILTER (THE FINE TUNING)
# ==============================================================================
# We use the master_engine again, but restricted to the audit_1 subset.
audit3, engine3 = plot_walk_forward_analyzer(
    engine=master_engine1,  # <-- Reuse engine1
    universe_subset=selected_tickers_1st_filter,  # <--- The 7 candidates from 1st run
    default_start_date=audit1[0].decision_date,  # <--- Perfectly sync'd date
    default_strategy="Sharpe (TRP)",  # Faster, high-conviction metric
    default_lookback=10,  # Short-term timing window
    default_rank_end=10,  # Final 10 tradeable tickers
    debug=True,  # Keep enabled for verification
)

FigureWidget({
    'data': [{'line': {'width': 2},
              'name': 'WBD',
              'type': 'scatter',
              'uid': 'b7ef6340-39f4-447f-b90d-303caa792b87',
              'visible': True,
              'x': array([datetime.datetime(2025, 11, 25, 0, 0),
                          datetime.datetime(2025, 11, 26, 0, 0),
                          datetime.datetime(2025, 11, 28, 0, 0),
                          datetime.datetime(2025, 12, 1, 0, 0),
                          datetime.datetime(2025, 12, 2, 0, 0),
                          datetime.datetime(2025, 12, 3, 0, 0),
                          datetime.datetime(2025, 12, 4, 0, 0),
                          datetime.datetime(2025, 12, 5, 0, 0),
                          datetime.datetime(2025, 12, 8, 0, 0),
                          datetime.datetime(2025, 12, 9, 0, 0),
                          datetime.datetime(2025, 12, 10, 0, 0),
                          datetime.datetime(2025, 12, 11, 0, 0),
                      

In [ ]:
selected_tickers_univ_subset_filter = audit3[0].tickers
print(
    f"selected_tickers_1st_filter count {len(selected_tickers_1st_filter)}: {selected_tickers_1st_filter}"
)
print(
    f"selected_tickers_univ_subset_filter count {len(selected_tickers_univ_subset_filter)}: {selected_tickers_univ_subset_filter}"
)

only_in_selected_tickers_univ_subset_filter = list(
    set(selected_tickers_univ_subset_filter) - set(selected_tickers_1st_filter)
)

print(
    f"\nWe asked for 10 tickers, it returned 7 tickers but INCLUDED a ticker that DID NOT passed the 1st filter"
)
print(f"This is WRONG behavior")
print("\n")
print(f"*" * 78)
print(
    f"only_in_selected_tickers_univ_subset_filter count {len(only_in_selected_tickers_univ_subset_filter)}: {only_in_selected_tickers_univ_subset_filter}"
)
print(f"*" * 78)

selected_tickers_1st_filter count 101: ['SGOV', 'SHV', 'BIL', 'MINT', 'USFR', 'BOXX', 'PULS', 'JPST', 'TD', 'VGSH', 'SHY', 'GLDM', 'SGOL', 'IAU', 'GLD', 'AU', 'JAAA', 'SLV', 'EWY', 'PSLV', 'STX', 'WDC', 'BTI', 'B', 'GDX', 'KGC', 'GDXJ', 'VCSH', 'SRLN', 'BKLN', 'NEM', 'CIEN', 'EFV', 'BSV', 'IGSB', 'INSM', 'LITE', 'BCS', 'HL', 'RGTI', 'WBD', 'IAG', 'GLW', 'CLS', 'AEM', 'MU', 'AVGO', 'APH', 'LRCX', 'EZU', 'GOOGL', 'GOOG', 'CAH', 'BBIO', 'QBTS', 'PAAS', 'SPDW', 'VEA', 'ITUB', 'VEU', 'KTOS', 'HOOD', 'IXUS', 'VXUS', 'GH', 'FNV', 'SCHF', 'EEM', 'IEMG', 'BE', 'WPM', 'TEL', 'HII', 'JNJ', 'VGK', 'KLAC', 'RTX', 'RY', 'TPR', 'SPIB', 'AG', 'BABA', 'EFA', 'MP', 'IEFA', 'NXT', 'BK', 'BBD', 'IDXX', 'GE', 'FIX', 'SGI', 'OKLO', 'PLTR', 'WELL', 'RPRX', 'CAT', 'ATI', 'CHRW', 'GSK', 'SPY']
selected_tickers_univ_subset_filter count 10: ['SHV', 'SGOV', 'CFLT', 'BIL', 'MCHP', 'MINT', 'STM', 'WBD', 'TEVA', 'TD']

We asked for 10 tickers, it returned 7 tickers but INCLUDED a ticker that DID NOT passed the 1st f

In [ ]:
res3 = visualize_audit_structure(audit3)

🔍 HIGH-TRANSPARENCY AUDIT MAP
[  0] 📂 audit_pack (len=1)
[  1]   📦 index_0 (EngineOutput)
[  2]     📈 portfolio_series (shape=(17,))
[  3]     📈 benchmark_series (shape=(17,))
[  4]     🧮 normalized_plot_data (shape=(17, 10))
[  5]     📂 tickers (len=10)
[  6]       📄 index_0 (str)
[  7]       📄 index_1 (str)
[  8]       📄 index_2 (str)
[  9]       📄 index_3 (str)
[ 10]       📄 index_4 (str)
[ 11]       📄 index_5 (str)
[ 12]       📄 index_6 (str)
[ 13]       📄 index_7 (str)
[ 14]       📄 index_8 (str)
[ 15]       📄 index_9 (str)
[ 16]     📈 initial_weights (shape=(10,))
[ 17]     📂 perf_metrics (len=24)
[ 18]       🔢 full_p_gain (float)
[ 19]       🔢 full_p_sharpe (float)
[ 20]       🔢 full_p_sharpe_atrp (float)
[ 21]       🔢 full_p_sharpe_trp (float)
[ 22]       🔢 lookback_p_gain (float)
[ 23]       🔢 lookback_p_sharpe (float)
[ 24]       🔢 lookback_p_sharpe_atrp (float)
[ 25]       🔢 lookback_p_sharpe_trp (float)
[ 26]       🔢 holding_p_gain (float)
[ 27]       🔢 holding_p_sharpe (fl

In [ ]:
verify_engine_results_short_form(audit_pack=audit3, engine=master_engine1)

🕵️  STARTING MULTI-LAYER AUDIT: Sharpe (TRP) @ 2025-12-10

*************************************************************************************
⚠️  ASSUMPTION: Verification logic is independent, but trusts Engine source DataFrames
   (engine.features_df, engine.df_close, and debug['portfolio_raw_components'])
*************************************************************************************


LAYER 1: SURVIVAL  | Universe: 1586 -> Survivors: 940 | ✅ PASS
LAYER 2: SELECTION | Strategy: Sharpe (TRP) | Selection Match: ✅ PASS
LAYER 3: PERFORMANCE | Holding Gain: -0.0144 vs Manual: -0.0144 | ✅ PASS


,Ticker,Engine,Manual,Delta,Match
Rank,,,,,
1,SHV,0.789797,0.789797,0.00000000,True
2,SGOV,0.753769,0.753769,0.00000000,True
3,CFLT,0.726028,0.726028,0.00000000,True
4,BIL,0.689552,0.689552,0.00000000,True
5,MCHP,0.670547,0.670547,0.00000000,True
6,MINT,0.667676,0.667676,0.00000000,True
7,STM,0.666708,0.666708,0.00000000,True
8,WBD,0.646641,0.646641,0.00000000,True
9,TEVA,0.635322,0.635322,0.00000000,True


,Engine,Manual,Delta
Metric,,,
Holding Gain,-0.01435532,-0.01435532,0.00000000
Holding Sharpe,-9.95117642,-9.95117642,0.00000000
Sharpe (ATRP),-0.17467580,-0.17467580,0.00000000


In [ ]:
verify_engine_results_long_form(audit_pack=audit3, engine=master_engine1)

MODULAR EVENT-DRIVEN AUDIT -- START
📝 FINAL INTEGRITY REPORT (MODULAR)


Period                     Full Holding Lookback
Entity    Metric                                
Benchmark Gain           ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe         ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe (ATRP)  ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe (TRP)   ✅ PASS  ✅ PASS   ✅ PASS
Group     Gain           ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe         ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe (ATRP)  ✅ PASS  ✅ PASS   ✅ PASS
          Sharpe (TRP)   ✅ PASS  ✅ PASS   ✅ PASS

,Entity,Period,Metric,Engine,Manual,Delta,Status
0,Group,Full,Gain,0.13089039,0.13089039,0.00000000,✅ PASS
1,Benchmark,Full,Gain,0.00214849,0.00214849,0.00000000,✅ PASS
2,Group,Full,Sharpe,10.24247659,10.24247659,0.00000000,✅ PASS
3,Benchmark,Full,Sharpe,0.41857488,0.41857488,0.00000000,✅ PASS
4,Group,Full,Sharpe (ATRP),0.40147494,0.40147494,0.00000000,✅ PASS
5,Benchmark,Full,Sharpe (ATRP),0.01277000,0.01277000,0.00000000,✅ PASS
6,Group,Full,Sharpe (TRP),0.41837814,0.41837814,0.00000000,✅ PASS
7,Benchmark,Full,Sharpe (TRP),0.01679050,0.01679050,0.00000000,✅ PASS
8,Group,Lookback,Gain,0.14502319,0.14502319,0.00000000,✅ PASS
9,Benchmark,Lookback,Gain,0.01859201,0.01859201,0.00000000,✅ PASS



SURVIVAL AUDIT -- START
Survival Integrity: ✅ PASS (Engine: 940 vs Manual: 940)

UNIVERSAL SELECTION AUDIT -- START


,Ticker,Engine_Score,Audit_Score,Delta,Status
Rank,,,,,
1,SHV,0.78979667,0.78979667,0.00000000,✅ PASS
2,SGOV,0.75376887,0.75376887,0.00000000,✅ PASS
3,CFLT,0.72602759,0.72602759,0.00000000,✅ PASS
4,BIL,0.68955244,0.68955244,0.00000000,✅ PASS
5,MCHP,0.67054680,0.67054680,0.00000000,✅ PASS
6,MINT,0.66767634,0.66767634,0.00000000,✅ PASS
7,STM,0.66670752,0.66670752,0.00000000,✅ PASS
8,WBD,0.64664101,0.64664101,0.00000000,✅ PASS
9,TEVA,0.63532151,0.63532151,0.00000000,✅ PASS
